In [2]:
from scipy.stats import kstest, shapiro, ttest_ind, mannwhitneyu, chi2_contingency, fisher_exact
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.multitest import multipletests
import shap
from nri_f import nri
from AUC_CI import auc_ci
from DCA import net_benefit_model, net_benefit_all, plot_DCA
from keras.models import load_model
import joblib
from hyperopt import Trials
from hyperopt import hp
from shaphypetune import BoostSearch, BoostRFE, BoostRFA, BoostBoruta
from copy import copy
from Delong_test import DelongTest
import seaborn as sns
from sklearn.utils import shuffle
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.calibration import calibration_curve
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, validation_curve, learning_curve
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, KBinsDiscretizer
from pandas.plotting import scatter_matrix
import numpy as np
from sklearn import metrics, datasets, clone
from pandas import set_option
from pandas import read_csv
from sklearn.metrics import roc_curve, auc, precision_recall_curve, precision_score, make_scorer, log_loss, roc_auc_score, mean_squared_error
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, RFECV, SelectFromModel
from matplotlib.ticker import MultipleLocator
from matplotlib import pyplot
import waterfall
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
from feature_selector import FeatureSelector
import feature_selector as selector
import pandas as pd
import matplotlib
from statsmodels.tools import add_constant
import statsmodels.stats.proportion as sm
from statsmodels.formula.api import logit
from scipy.stats import norm
import scipy.stats as stats
from math import sqrt
import math
from itertools import combinations
import warnings
import winsound
duration = 500
frequency = 440
font_set = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=12)

warnings.filterwarnings('ignore')

In [ ]:
df_new_filepath = r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\non_split_chuyichangzhihou_all.xlsx'
df_new = pd.read_excel(df_new_filepath, sheet_name=0)

In [ ]:
y = df_new['label']
x = df_new

test_size = 0.3
seed = 42

x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=test_size,
    stratify=y,
    shuffle=True,
    random_state=seed)

print(x_train.shape)
print(x_test.shape)

x_train.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_train_nonstd.xlsx', index=False)
x_test.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_test_nonstd.xlsx', index=False)
print(' ')

In [ ]:
non_std_names = ['label', 'Name', 'Nodules',	'Shape',
                 'Margin',	'Cystic',	'ETE',	'Enhancement', 'TI_RADS_score']
colNames = x_train.columns
std_names = colNames.drop(non_std_names)

sc = StandardScaler()
sc.fit(x_train[std_names])

x_train_sc = sc.transform(x_train[std_names])
x_train[std_names] = pd.DataFrame(
    x_train_sc, columns=std_names, index=x_train.index)
x_train.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_train_std_all.xlsx', index=False)

x_test_sc = sc.transform(x_test[std_names])
x_test[std_names] = pd.DataFrame(
    x_test_sc, columns=std_names, index=x_test.index)
x_test.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_test_std_all.xlsx', index=False)

In [ ]:
tradition_labes = ['MAD_mm', 'Nodules',	'Shape', 'Margin',	'Cystic',	'ETE',	'Enhancement',	'T1_rCNR',	'T1C_rCNR',
                   'T1C_increase',	'T2_rCNR',	'B800_rCNR',	'B2000_rCNR',	'ADC_lesion',	'ADC_normal',	'ADC_rCNR',	'TI_RADS_score']

non_radiomics_labels = ['label', 'Name', 'MAD_mm', 'Nodules',	'Shape', 'Margin',	'Cystic',	'ETE',	'Enhancement',	'T1_rCNR',
                        'T1C_rCNR', 'T1C_increase',	'T2_rCNR',	'B800_rCNR',	'B2000_rCNR',	'ADC_lesion',	'ADC_normal',	'ADC_rCNR',	'TI_RADS_score']

radiomics_labels = colNames.drop(non_radiomics_labels)

In [ ]:
x_trainT = x_train.loc[:, tradition_labes]
x_testT = x_test.loc[:, tradition_labes]
print(x_trainT.shape)
print(x_testT.shape)
print(' ')

x_train_MRI = x_train[non_radiomics_labels]
x_train_MRI.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_train_MRI.xlsx', index=False)
x_test_MRI = x_test[non_radiomics_labels]
x_test_MRI.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_test_MRI.xlsx', index=False)

In [ ]:
def normality_test(df, cols):
    results = []
    for col in cols:

        group0 = df[df['label'] == 0][col]
        group1 = df[df['label'] == 1][col]

        _, p0 = kstest(group0, 'norm')
        _, p1 = kstest(group1, 'norm')

        if p0 > 0.05 and p1 > 0.05:

            t, p = ttest_ind(group0, group1)
            test_type = 't-test'
        else:

            t, p = mannwhitneyu(group0, group1)
            test_type = 'Mann-Whitney'

        results.append((col, test_type, t, p))

    return pd.DataFrame(results, columns=['variable', 'test_type', 'statistic', 'p_value'])


def categorical_test(df, cols):
    results = []
    for col in cols:

        table = pd.crosstab(df[col], df['label'])
        if table.values.min() >= 5:
            chi2, p, _, _ = chi2_contingency(table)
            test_type = 'chi-square'
        else:
            _, p = fisher_exact(table)
            test_type = 'Fisher'

        results.append((col, test_type, None, p))

    return pd.DataFrame(results, columns=['variable', 'test_type', 'statistic', 'p_value'])


numeric_cols = ['MAD_mm', 'T1_rCNR', 'T1C_rCNR', 'T1C_increase', 'T2_rCNR',
                'B800_rCNR', 'B2000_rCNR', 'ADC_lesion', 'ADC_normal', 'ADC_rCNR', 'TI_RADS_score']
normality_res = normality_test(x_train_MRI, numeric_cols)


categorical_cols = ['Nodules', 'Shape',
                    'Margin', 'Cystic', 'ETE', 'Enhancement']
categorical_res = categorical_test(x_train_MRI, categorical_cols)


significant_vars = list(normality_res[normality_res['p_value'] < 0.05]['variable']) + \
    list(categorical_res[categorical_res['p_value'] < 0.05]['variable'])


formula = 'label ~ ' + ' + '.join(significant_vars)
model1 = logit(formula=formula, data=x_train_MRI).fit()
print("\n", model1.summary())

independent_vars = list(model1.pvalues[model1.pvalues < 0.05].index)
x_train_MRI = add_constant(x_train_MRI)
if 'Intercept' in independent_vars:
    independent_vars.remove('Intercept')
formula = 'label ~ ' + ' + '.join(independent_vars)
clf_MRI = logit(formula=formula, data=x_train_MRI).fit()
print("\n", clf_MRI.summary())

In [ ]:
# clf_MRI_train_auc
y_pred_proba = clf_MRI.predict(x_train_MRI[independent_vars])
clf_MRI_train_auc = roc_auc_score(y_train, y_pred_proba)
print("clf_MRI_train_auc:", clf_MRI_train_auc)
clf_MRI_train_auc_CI = auc_ci(y_true=y_train, y_score=clf_MRI.predict(
    x_train_MRI[independent_vars]), positive=1)
print("clf_MRI_train_auc_CI:", clf_MRI_train_auc_CI)

y_pred_proba = clf_MRI.predict(x_train_MRI[independent_vars])
y_pred = (y_pred_proba >= 0.5).astype(int)

# clf_MRI Logit Training set confusion_matrix
print("clf_MRI Logit Training set confusion_matrix：")
tn, fp, fn, tp = confusion_matrix(y_true=y_train, y_pred=y_pred).ravel()
print("Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# clf_MRI Training set Sensitivity
sensitivity = tp / (tp + fn)
print("clf_MRI Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("clf_MRI Training set 95% CI (sensitivity):", ci_sensitivity)

# clf_MRI Training set Specificity
specificity = tn / (tn + fp)
print("clf_MRI Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("clf_MRI Training set 95% CI (specificity):", ci_specificity)

# clf_MRI Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("clf_MRI Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("clf_MRI Training set 95% CI (accuracy):", ci_accuracy)

print("\n")

# clf_MRI_test_auc
y_pred_proba = clf_MRI.predict(x_test_MRI[independent_vars])
clf_MRI_test_auc = roc_auc_score(y_test, y_pred_proba)
print("clf_MRI_test_auc:", clf_MRI_test_auc)
clf_MRI_test_auc_CI = auc_ci(y_true=y_test, y_score=clf_MRI.predict(
    x_test_MRI[independent_vars]), positive=1)
print("clf_MRI_test_auc_CI:", clf_MRI_test_auc_CI)

y_pred_proba = clf_MRI.predict(x_test_MRI[independent_vars])
y_pred = (y_pred_proba >= 0.5).astype(int)

# clf_MRI Logit Test set confusion_matrix
print("clf_MRI Logit Test set confusion_matrix：")
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
print("Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# clf_MRI Test set Sensitivity
sensitivity = tp / (tp + fn)
print("clf_MRI Test set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("clf_MRI Test set 95% CI (sensitivity):", ci_sensitivity)

# clf_MRI Test set Specificity
specificity = tn / (tn + fp)
print("clf_MRI Test set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("clf_MRI Test set 95% CI (specificity):", ci_specificity)

# clf_MRI Test set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("clf_MRI Test set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("clf_MRI Test set 95% CI (accuracy):", ci_accuracy)

In [ ]:
# clf_MRI testing set ROC curve
def auc_MRI_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_MRI_CI_L = auc_MRI_ci_lower(y_true=y_test, y_score=clf_MRI.predict(
    x_test_MRI[independent_vars]), positive=1)
# print("MRI_AUC_95%CI_lower:", AUC_MRI_CI_L)

# auc_MRI 95% CI_upper：


def auc_MRI_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_MRI_CI_U = auc_MRI_ci_upper(y_true=y_test, y_score=clf_MRI.predict(
    x_test_MRI[independent_vars]), positive=1)
# print("MRI_AUC_95%CI_upper:", AUC_MRI_CI_U)

# ROC
y_probs = clf_MRI.predict(x_test_MRI[independent_vars])
fpr, tpr, thresholds = roc_curve(y_test, y_probs, pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % clf_MRI_test_auc + '(%0.2f' % AUC_MRI_CI_L + '-' + '%0.2f)' % AUC_MRI_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Logit ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Logit_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# clf_MRI calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_MRI.predict(x_test_MRI[independent_vars]), n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.figure(dpi=300)
plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='Logit Model')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean predicted value", fontsize=12)
plt.ylabel("Fraction of positives", fontsize=12)
plt.title("Logit Model Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Logit_cal.png', dpi=300)
plt.show()

In [ ]:
# clf_MRI DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_MRI.predict(x_test_MRI[independent_vars])
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('Logit Model Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Logit_DCA.png', dpi=300)
plt.show()

In [ ]:
# FeatureSelector_1
fs = FeatureSelector(data=x_train[radiomics_labels], labels=y_train)

# correlation_threshold is 0.8
fs.identify_collinear(correlation_threshold=0.8)
collinear_features = fs.ops['collinear']
print('collinear_features:', collinear_features[:10])
print('collinear_features:', fs.record_collinear.head())

In [ ]:
# FeatureSelector_2
fs.identify_zero_importance(task='classification',
                            eval_metric='auc',
                            n_iterations=10,
                            early_stopping=True)

# list of zero importance features
zero_importance_features = fs.ops['zero_importance']
print('zero_importance_features:', zero_importance_features[:10])

# one hot features
one_hot_features = fs.one_hot_features
base_features = fs.base_features
print('There are %d original features' % len(base_features))
print('There are %d one-hot features' % len(one_hot_features))

fs.plot_feature_importances(threshold=0.99, plot_n=12)

winsound.Beep(frequency, duration)

In [ ]:
# FeatureSelector_3
fs.identify_low_importance(cumulative_importance=0.99)
low_importance_features = fs.ops['low_importance']
print('low_importance_features:', low_importance_features[:10])

selected_features = fs.remove(methods='all', keep_one_hot=False)
fs_names = selected_features.columns

print('selected_features；', selected_features.shape)
print('fs_names:', fs_names[:10])
print(x_train[radiomics_labels].shape[1])
print(selected_features.shape[1])

winsound.Beep(frequency, duration)

In [ ]:
selected_columns = non_radiomics_labels + list(fs_names)
x_train_fs = x_train[selected_columns]
x_test_fs = x_test[selected_columns]

print(x_train_fs.shape)
print(x_test_fs.shape)

x_train_step1 = x_train_fs
x_test_step1 = x_test_fs
print("x_train_step1：", x_train_step1.shape)
print("x_test_step1：", x_test_step1.shape)

x_train_step1 = x_train_fs
x_test_step1 = x_test_fs
print("x_train_step1：", x_train_step1.shape)
print("x_test_step1：", x_test_step1.shape)
x_trainT = x_train_step1.loc[:, tradition_labes]
x_testT = x_test_step1.loc[:, tradition_labes]
print(x_trainT.shape)
print(x_testT.shape)
print(' ')

x_train_MRI = x_train_step1[non_radiomics_labels]
x_train_MRI.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_train_MRI.xlsx', index=False)
x_test_MRI = x_test_step1[non_radiomics_labels]
x_test_MRI.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_test_MRI.xlsx', index=False)

x_train_radiomics = x_train_step1.loc[:, fs_names]
x_test_radiomics = x_test_step1.loc[:, fs_names]
print(x_train_radiomics.shape)
print(x_test_radiomics.shape)

radiomics_columns = ['label', 'Name'] + list(fs_names)
x_train_radiomics_name = x_train_step1.loc[:, radiomics_columns]
x_test_radiomics_name = x_test_step1.loc[:, radiomics_columns]
x_train_radiomics_name.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_train__radiomics.xlsx', index=False)
x_test_radiomics_name.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_test__radiomics.xlsx', index=False)

x_train_sc = x_train_radiomics
x_test_sc = x_test_radiomics

In [ ]:
# LASSO
lr = LogisticRegression(penalty='l1', max_iter=100000,
                        solver='liblinear', multi_class='ovr', random_state=42)
penaltys = ['l1']
Cs = np.logspace(-1, 1, 50)
tuned_parameters = dict(penalty=penaltys, C=Cs)

grid = GridSearchCV(lr, tuned_parameters, return_train_score=True, cv=10)

grid.fit(x_train_sc, y_train)
print('clf.best_score_:', grid.best_score_)
print('grid.best_params_', grid.best_params_)
print(' clf.best_estimator_:', grid.best_estimator_)
regr_cv = grid.best_estimator_

test_means = grid.cv_results_['mean_test_score']
test_stds = grid.cv_results_['std_test_score']
train_means = grid.cv_results_['mean_train_score']
train_stds = grid.cv_results_['std_train_score']
# plot results
n_Cs = len(Cs)
number_penaltys = len(penaltys)
test_scores = np.array(test_means).reshape(n_Cs, number_penaltys)
train_scores = np.array(train_means).reshape(n_Cs, number_penaltys)
test_stds = np.array(test_stds).reshape(n_Cs, number_penaltys)
train_stds = np.array(train_stds).reshape(n_Cs, number_penaltys)

x_axis = np.log10(Cs)
for i, value in enumerate(penaltys):
    plt.errorbar(
        x_axis, test_scores[:, i], yerr=test_stds[:, i], label=penaltys[i] + ' Test')
    plt.errorbar(
        x_axis, train_scores[:, i], yerr=train_stds[:, i], label=penaltys[i] + ' Train')
plt.legend()
plt.xlabel('C')
plt.axvline(grid.best_params_['C'], color='black', ls="--")
plt.ylabel('Accuary')
# plt.savefig('LogisticGridSearchCV_C.png')
pyplot.show()

lr = LogisticRegression(penalty='l1',
                        C=grid.best_params_['C'],
                        solver='liblinear',
                        multi_class='ovr',
                        random_state=42)

lr.fit(x_train_sc, y_train)
print('Train accuracy:', lr.score(x_train_sc, y_train))
print('Test accuracy:', lr.score(x_test_sc, y_test))
# print('lr.coef_:', lr.coef_)
# print('lr.intercept_:', lr.intercept_)


# SelectFromModel
threshold = np.percentile(np.abs(lr.coef_), 75)
selector = SelectFromModel(estimator=lr, threshold=None, prefit=False)
selector.fit(x_train_sc, y_train)
x_selected = selector.transform(x_train_sc)
x_selected_test = selector.transform(x_test_sc)

print('\n', 'Number of features that meet this threshold criterion:',
      x_selected.shape[1], '\n')

In [ ]:
feat_labels = fs_names
index = lr.coef_[lr.coef_ != 0]
index = index.ravel()
indices = np.argsort(index)[::-1]
res_1, res_2 = [], []
for f in range(x_selected.shape[1]):
    index_ = index[indices[f]]
    res_1.append(index_)
    feat_labels_ = feat_labels[indices[f]]
    res_2.append(feat_labels_)
coef_series = pd.Series(res_1, index=np.array(res_2))
pd.set_option('display.max_rows', None)

print(coef_series)

# coef_series to DataFrame
coef_series_list = pd.concat([pd.Series(res_2), pd.Series(res_1)], axis=1)
coef_series_list.columns = ['feature', 'coef']
# coef_series_list to excel
coef_series_list.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\coef_series_list.xlsx', index=False)

fig = plt.figure(dpi=300)
ax = plt.subplot(111)
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
coef_series.plot(kind="bar", color='k', grid=True, alpha=0.5)
plt.xticks(rotation=30, ha='right', va='top', fontsize=4)  # fontsize=5
plt.title("Coefficients in Model")
plt.subplots_adjust(left=0.2, bottom=0.5, right=0.95, top=0.95)
fig.tight_layout()
plt.show()

selected_column_names = coef_series.index
x_selected = pd.DataFrame(x_train_sc, columns=selected_column_names)
x_selected_test = pd.DataFrame(x_test_sc, columns=selected_column_names)

x_selected.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_selected_train.xlsx', index=False)
x_selected_test.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_selected_test.xlsx', index=False)

In [ ]:
fig = plt.figure(dpi=300)
ax = plt.subplot(111)

weights, params = [], []
for c in np.arange(-4., 6.):
    lr = LogisticRegression(penalty='l1', C=10. ** c, solver='liblinear',
                            multi_class='ovr', random_state=42)
    lr.fit(x_train_sc, y_train)
    weights.append(lr.coef_[0])
    params.append(10 ** c)

weights = np.array(weights)

for column in range(weights.shape[1]):
    plt.plot(params, weights[:, column],
             label=x.columns[column])
plt.axvline(grid.best_params_['C'], color='black', ls="--")
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.xlim([10 ** (-5), 10 ** 5])
plt.ylabel('Coefficients Weights', fontproperties=font_set)
plt.xlabel('C')
plt.xscale('log')
# plt.legend(loc='upper left')
# ax.legend(loc='upper center',
#           bbox_to_anchor=(1.38, 1.03),
#           ncol=1, fancybox=True)
# plt.savefig('images/04_07.png', dpi=300,
#            bbox_inches='tight', pad_inches=0.2)
plt.show()

In [ ]:
MSEs_mean = grid.cv_results_['mean_test_score']
MSEs_std = grid.cv_results_['std_test_score']

fig = plt.figure(dpi=300)
ax = plt.subplot(111)
plt.errorbar(np.log10(1 / Cs), MSEs_mean, yerr=MSEs_std, fmt="o", ms=3,
             mfc="r", mec="r", ecolor="lightblue", elinewidth=2, capsize=4, capthick=1)
plt.semilogx()
plt.axvline(grid.best_params_['C'], color="black", ls="--")
plt.xlabel("C")
# plt.xlim([10 ** (-1), 10 ** 1])
plt.ylabel("MSE")
ax = plt.gca()
y_major_locator = MultipleLocator(0.05)
ax.yaxis.set_major_locator(y_major_locator)
plt.show()

In [ ]:
selected_features_filepath = r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\coef_series_list.xlsx'
selected_features = pd.read_excel(selected_features_filepath, sheet_name=0)
print(selected_features)

selected_features.columns = ['names', 'coef']
selected_column_names = selected_features['names']
print(selected_column_names)

x_selected = pd.DataFrame(x_train, columns=selected_column_names)
x_selected_test = pd.DataFrame(x_test, columns=selected_column_names)

x_selected.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_selected_train.xlsx', index=False)
x_selected_test.to_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\x_selected_test.xlsx', index=False)

In [ ]:
num_folds = 10
seed = 42
scoring = 'accuracy'

In [ ]:
# LR model
lr = LogisticRegression(C=grid.best_params_['C'], penalty='l1', max_iter=100000, solver='liblinear', multi_class='ovr', random_state=42)
lr.fit(x_selected, y_train)

# LR AUC in Training set
LR_Training_AUC = roc_auc_score(y_train,lr.predict_proba(x_selected)[:, 1])
print("LR_Training_AUC:", LR_Training_AUC)
LR_Training_AUC_CI = auc_ci(y_true=y_train, y_score=lr.predict_proba(x_selected)[:, 1], positive=1)
print("LR_Training_AUC_95%CI:", LR_Training_AUC_CI)

# LR confusion_matrix in Training set
print("LR confusion_matrix in Training set：")
tn, fp, fn, tp = confusion_matrix(y_true = y_train, y_pred = lr.predict(x_selected)).ravel()
print("Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# LR Training set Sensitivity
sensitivity = tp / (tp + fn)
print("LR Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("LR Training set 95% CI (sensitivity):", ci_sensitivity)

# LR Training set Specificity
specificity = tn / (tn + fp)
print("LR Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples=n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("LR Training set 95% CI (specificity):", ci_specificity)

# LR Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("LR Training set accuracy:", accuracy)
n_correct = tn + tp 
n_samples= tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("LR Training set 95% CI (accuracy):", ci_accuracy)

print("\n")

# LR AUC in Testing set
LR_Testing_AUC = roc_auc_score(y_test,lr.predict_proba(x_selected_test)[:, 1])
print("LR_Testing_AUC:", LR_Testing_AUC)
LR_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=lr.predict_proba(x_selected_test)[:, 1], positive=1)
print("LR_Testing_AUC_95%CI:", LR_Testing_AUC_CI)

# LR confusion_matrix in Testing set
print("LR confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = lr.predict(x_selected_test)).ravel()
print("Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# LR Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("LR Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("LR Testing set 95% CI (sensitivity):", ci_sensitivity)

# LR Testing set Specificity
specificity = tn / (tn + fp)
print("LR Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("LR Testing set 95% CI (specificity):", ci_specificity)

# LR Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("LR Testing set accuracy:", accuracy)
n_correct = tn + tp 
n_samples= tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("LR Testing set 95% CI (accuracy):", ci_accuracy)

# LR testing set ROC curve

# auc_lr 95% CI_lower：
def auc_lr_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) + (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_LR_CI_L = auc_lr_ci_lower(y_true=y_test, y_score=lr.predict_proba(x_selected_test)[:, 1], positive=1)
# print("LR_AUC_95%CI_lower:", AUC_LR_CI_L)


# auc_lr 95% CI_upper：
def auc_lr_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) + (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_LR_CI_U = auc_lr_ci_upper(y_true=y_test, y_score=lr.predict_proba(x_selected_test)[:, 1], positive=1)
# print("LR_AUC_95%CI_upper:", AUC_LR_CI_U)

# ROC
y_probs = lr.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % LR_Testing_AUC + '(%0.2f' % AUC_LR_CI_L + '-' + '%0.2f)' % AUC_LR_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('LR ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LR_ROC.png', dpi = 300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# LR SHAP
explainer = shap.LinearExplainer(lr, x_selected)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="LR SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'LR_shap.png')
plt.close(fig)

In [ ]:
# LR calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, lr.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='LR Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("LR Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LR_cal.png', dpi=300)
plt.show()

In [ ]:
# LR_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = lr.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('LR Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LR_DCA.png', dpi=300)
plt.show()

In [ ]:
# KNN tuning
neighbors = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
param_grid = dict(n_neighbors=neighbors)
KNN = KNeighborsClassifier()
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
grid = GridSearchCV(estimator=KNN, param_grid=param_grid,
                    scoring=scoring, cv=kfold)
grid.fit(x_train_sc, y_train)
print('\n', "KNN Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# KNN fit training set
clf_KNN = grid.best_estimator_
clf_KNN.fit(x_selected, y_train)

# KNN AUC in Training set
KNN_Training_AUC = roc_auc_score(
    y_train, clf_KNN.predict_proba(x_selected)[:, 1])
print("KNN_Training_AUC:", KNN_Training_AUC)
KNN_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=clf_KNN.predict_proba(x_selected)[:, 1], positive=1)
print("KNN_Training_AUC_95%CI:", KNN_Training_AUC_CI)

# KNN confusion_matrix in Training set
print("KNN confusion_matrix in Training set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=clf_KNN.predict(x_selected)).ravel()
print("Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# KNN Training set Sensitivity
sensitivity = tp / (tp + fn)
print("KNN Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("KNN Training set 95% CI (sensitivity):", ci_sensitivity)

# KNN Training set Specificity
specificity = tn / (tn + fp)
print("KNN Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("KNN Training set 95% CI (specificity):", ci_specificity)

# KNN Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("KNN Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("KNN Training set 95% CI (accuracy):", ci_accuracy)

print("\n")

# KNN AUC in Testing set
KNN_Testing_AUC = roc_auc_score(
    y_test, clf_KNN.predict_proba(x_selected_test)[:, 1])
print("KNN_Testing_AUC:", KNN_Testing_AUC)
KNN_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_KNN.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("KNN_Testing_AUC_95%CI:", KNN_Testing_AUC_CI)

# KNN confusion_matrix in Testing set
print("KNN confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=clf_KNN.predict(x_selected_test)).ravel()
print("Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# KNN Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("KNN Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("KNN Testing set 95% CI (sensitivity):", ci_sensitivity)

# KNN Testing set Specificity
specificity = tn / (tn + fp)
print("KNN Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("KNN Testing set 95% CI (specificity):", ci_specificity)

# KNN Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("KNN Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("KNN Testing set 95% CI (accuracy):", ci_accuracy)

# KNN Testing set ROC

# auc_KNN 95% CI_lower：
def auc_KNN_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_KNN_CI_L = auc_KNN_ci_lower(
    y_true=y_test, y_score=clf_KNN.predict_proba(x_selected_test)[:, 1], positive=1)
print("KNN_AUC_95%CI_lower:", AUC_KNN_CI_L)


# auc_KNN 95% CI_upper：
def auc_KNN_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_KNN_CI_U = auc_KNN_ci_upper(
    y_true=y_test, y_score=clf_KNN.predict_proba(x_selected_test)[:, 1], positive=1)
print("KNN_AUC_95%CI_upper:", AUC_KNN_CI_U)

# ROC
y_probs = clf_KNN.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % KNN_Testing_AUC + '(%0.2f' % AUC_KNN_CI_L + '-' + '%0.2f)' % AUC_KNN_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('KNN ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\KNN_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# KNN SHAP
explainer = shap.KernelExplainer(clf_KNN.predict, x_selected)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="KNN SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'KNN_shap.png')
plt.close(fig)

In [ ]:
# KNN calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_KNN.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='KNN Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("KNN Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\KNN_cal.png', dpi=300)
plt.show()

In [ ]:
# KNN_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_KNN.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('KNN Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\KNN_DCA.png', dpi=300)
plt.show()

In [ ]:
# svm tuning
Cs = [0.001, 0.005, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.7,
      0.9, 1.0, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 3.0]
gammas = np.logspace(-4, 1, 10, base=2)
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']

param_grid = dict(C=Cs, kernel=kernel_values, gamma=gammas)
model = SVC(probability=True)
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    scoring=scoring, cv=kfold)
grid.fit(x_train_sc, y_train)
print('\n', "SVM Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# SVM fit training set
clf_SVM = grid.best_estimator_
clf_SVM.fit(x_selected, y_train)

# SVM AUC in Training set
SVM_Training_AUC = roc_auc_score(
    y_train, clf_SVM.predict_proba(x_selected)[:, 1])
print("SVM_Training_AUC:", SVM_Training_AUC)
SVM_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=clf_SVM.predict_proba(x_selected)[:, 1], positive=1)
print("SVM_Training_AUC_95%CI:", SVM_Training_AUC_CI)

# SVM confusion_matrix in Training set
print("SVM confusion_matrix in Training set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=clf_SVM.predict(x_selected)).ravel()
print("SVM Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# SVM Training set Sensitivity
sensitivity = tp / (tp + fn)
print("SVM Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("SVM Training set 95% CI (sensitivity):", ci_sensitivity)

# SVM Training set Specificity
specificity = tn / (tn + fp)
print("SVM Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("SVM Training set 95% CI (specificity):", ci_specificity)

# SVM Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("SVM Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("SVM Training set 95% CI (accuracy):", ci_accuracy)

print("\n")

# SVM AUC in Testing set
SVM_Testing_AUC = roc_auc_score(
    y_test, clf_SVM.predict_proba(x_selected_test)[:, 1])
print("SVM_Testing_AUC:", SVM_Testing_AUC)
SVM_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_SVM.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("SVM_Testing_AUC_95%CI:", SVM_Testing_AUC_CI)

# SVM confusion_matrix in Testing set
print("SVM confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=clf_SVM.predict(x_selected_test)).ravel()
print("SVM Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# SVM Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("SVM Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("SVM Testing set 95% CI (sensitivity):", ci_sensitivity)

# SVM Testing set Specificity
specificity = tn / (tn + fp)
print("SVM Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("SVM Testing set 95% CI (specificity):", ci_specificity)

# SVM Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("SVM Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("SVM Testing set 95% CI (accuracy):", ci_accuracy)

# SVM test set ROC

# auc_SVM 95% CI_lower：
def auc_SVM_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_SVM_CI_L = auc_SVM_ci_lower(
    y_true=y_test, y_score=clf_SVM.predict_proba(x_selected_test)[:, 1], positive=1)
# print("SVM_AUC_95%CI_lower:", AUC_SVM_CI_L)


# auc_SVM 95% CI_upper：
def auc_SVM_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_SVM_CI_U = auc_SVM_ci_upper(
    y_true=y_test, y_score=clf_SVM.predict_proba(x_selected_test)[:, 1], positive=1)
# print("SVM_AUC_95%CI_upper:", AUC_SVM_CI_U)

# ROC
y_probs = clf_SVM.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % SVM_Testing_AUC + '(%0.2f' % AUC_SVM_CI_L + '-' + '%0.2f)' % AUC_SVM_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('SVM ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\SVM_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# SVM SHAP
explainer = shap.KernelExplainer(clf_SVM.predict, x_selected)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="SVM SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'SVM_shap.png')
plt.close(fig)

In [ ]:
# SVM calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_SVM.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='SVM Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("SVM Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\SVM_Cal.png', dpi=300)
plt.show()

In [ ]:
# SVM_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_SVM.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('SVM Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\SVM_DCA.png', dpi=300)
plt.show()

In [ ]:
# LDA tuning
solvers = ['svd', 'lsqr', 'eigen']
shrinkages = [0.3, 0.5, 0.7, 0.9, 1.1]
param_grid = dict(solver=solvers, shrinkage=shrinkages)
LDA = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
grid = GridSearchCV(estimator=LDA, param_grid=param_grid,
                    scoring=scoring, cv=kfold)
grid.fit(x_train_sc, y_train)
# print('LDA的estimator.get_params():',grid.estimator.get_params().keys())
print('\n', "LDA Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# LDA fit training set
clf_LDA = grid.best_estimator_
clf_LDA.fit(x_selected, y_train)

# LDA AUC in Training set
LDA_Training_AUC = roc_auc_score(
    y_train, clf_LDA.predict_proba(x_selected)[:, 1])
print("LDA_Training_AUC:", LDA_Training_AUC)
LDA_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=clf_LDA.predict_proba(x_selected)[:, 1], positive=1)
print("LDA_Training_AUC_95%CI:", LDA_Training_AUC_CI)

# LDA confusion_matrix in Training set
print("LDA confusion_matrix in Training set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=clf_LDA.predict(x_selected)).ravel()
print("LDA Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# LDA Training set Sensitivity
sensitivity = tp / (tp + fn)
print("LDA Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("LDA Training set 95% CI (sensitivity):", ci_sensitivity)

# LDA Training set Specificity
specificity = tn / (tn + fp)
print("LDA Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("LDA Training set 95% CI (specificity):", ci_specificity)

# LDA Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("LDA Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("LDA Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# LDA AUC in Testing set
LDA_Testing_AUC = roc_auc_score(
    y_test, clf_LDA.predict_proba(x_selected_test)[:, 1])
print("LDA_Testing_AUC:", LDA_Testing_AUC)
LDA_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_LDA.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("LDA_Testing_AUC_95%CI:", LDA_Testing_AUC_CI)

# LDA confusion_matrix in Testing set
print("LDA confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=clf_LDA.predict(x_selected_test)).ravel()
print("LDA Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# LDA Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("LDA Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("LDA Testing set 95% CI (sensitivity):", ci_sensitivity)

# LDA Testing set Specificity
specificity = tn / (tn + fp)
print("LDA Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("LDA Testing set 95% CI (specificity):", ci_specificity)

# LDA Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("LDA Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("LDA Testing set 95% CI (accuracy):", ci_accuracy)

# LDA test ROC

# auc_LDA 95% CI_lower：
def auc_LDA_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_LDA_CI_L = auc_LDA_ci_lower(
    y_true=y_test, y_score=clf_LDA.predict_proba(x_selected_test)[:, 1], positive=1)
# print("LDA_AUC_95%CI_lower:", AUC_LDA_CI_L)

# auc_LDA 95% CI_upper：
def auc_LDA_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_LDA_CI_U = auc_LDA_ci_upper(
    y_true=y_test, y_score=clf_LDA.predict_proba(x_selected_test)[:, 1], positive=1)
# print("LDA_AUC_95%CI_upper:", AUC_LDA_CI_U)

# ROC
y_probs = clf_LDA.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % LDA_Testing_AUC + '(%0.2f' % AUC_LDA_CI_L + '-' + '%0.2f)' % AUC_LDA_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('LDA ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LDA_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# LDA SHAP
explainer = shap.LinearExplainer(clf_LDA, x_selected)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="LDA SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'LDA_shap.png')
plt.close(fig)

In [ ]:
# LDA calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_LDA.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='LDA Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("LDA Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LDA_Cal.png', dpi=300)
plt.show()

In [ ]:
# LDA_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_LDA.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0,1,0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig,ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('LDA Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LDA_DCA.png', dpi = 300)
plt.show()

In [ ]:
# GNB tuning

param_grid = dict(priors=[[0.68, 0.32], [0.7, 0.3], [0.8, 0.2]], var_smoothing=[1e-9, 1e-8, 1e-7], )
model = GaussianNB()
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid.fit(x_train_sc, y_train)
print('\n', "GNB Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# GNB fit train set
clf_GNB = grid.best_estimator_
clf_GNB.fit(x_selected, y_train)

# GNB AUC in Training set
GNB_Training_AUC = roc_auc_score(y_train, clf_GNB.predict_proba(x_selected)[:, 1])
print("GNB_Training_AUC:", GNB_Training_AUC)
GNB_Training_AUC_CI = auc_ci(y_true=y_train, y_score=clf_GNB.predict_proba(x_selected)[:, 1], positive=1)
print("GNB_Training_AUC_95%CI:", GNB_Training_AUC_CI)

# GNB confusion_matrix in Training set
print("GNB模型Training set中混淆矩阵：")
tn, fp, fn, tp = confusion_matrix(y_true = y_train, y_pred = clf_GNB.predict(x_selected)).ravel()
print("GNB Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# GNB Training set Sensitivity
sensitivity = tp / (tp + fn)
print("GNB Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("GNB Training set 95% CI (sensitivity):", ci_sensitivity)

# GNB Training set Specificity
specificity = tn / (tn + fp)
print("GNB Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples=n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("GNB Training set 95% CI (specificity):", ci_specificity)

# GNB Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("GNB Training set accuracy:", accuracy)
n_correct = tn + tp 
n_samples= tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("GNB Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# GNB AUC in Testing set
GNB_Testing_AUC = roc_auc_score(y_test,clf_GNB.predict_proba(x_selected_test)[:, 1])
print("GNB AUC in Testing set:", GNB_Testing_AUC)
GNB_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_GNB.predict_proba(x_selected_test)[:, 1], positive=1)
print("GNB AUC in Testing set 95%CI:", GNB_Testing_AUC_CI)

# GNB confusion_matrix in Testing set
print("GNB confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = clf_GNB.predict(x_selected_test)).ravel()
print("GNB Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# GNB Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("GNB Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("GNB Testing set 95% CI (sensitivity):", ci_sensitivity)

# GNB Testing set Specificity
specificity = tn / (tn + fp)
print("GNB Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("GNB Testing set 95% CI (specificity):", ci_specificity)

# GNB Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("GNB Testing set accuracy:", accuracy)
n_correct = tn + tp 
n_samples= tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("GNB Testing set 95% CI (accuracy):", ci_accuracy)

# GNB test ROC

# auc_GNB 95% CI_lower：
def auc_GNB_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) + (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)

AUC_GNB_CI_L = auc_GNB_ci_lower(y_true=y_test, y_score=clf_GNB.predict_proba(x_selected_test)[:, 1], positive=1)
# print("GNB_AUC_95%CI_lower:", AUC_GNB_CI_L)

# auc_GNB 95% CI_upper：
def auc_GNB_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) + (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)

AUC_GNB_CI_U = auc_GNB_ci_upper(y_true=y_test, y_score=clf_GNB.predict_proba(x_selected_test)[:, 1], positive=1)
# print("GNB_AUC_95%CI_upper:", AUC_GNB_CI_U)

# ROC
y_probs = clf_GNB.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % GNB_Testing_AUC + '(%0.2f' % AUC_GNB_CI_L + '-' + '%0.2f)' % AUC_GNB_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('GNB ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\GNB_ROC.png', dpi = 300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# GNB SHAP
explainer = shap.KernelExplainer(clf_GNB.predict, x_selected)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="GNB SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'GNB_shap.png')
plt.close(fig)

In [ ]:
# GNB calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_GNB.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='GNB Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean predicted value", fontsize=12)
plt.ylabel("Fraction of positives", fontsize=12)
plt.title("GNB Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\GNB_cal.png', dpi=300)
plt.show()

In [ ]:
# GNB_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_GNB.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('GNB Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\GNB_DCA.png', dpi=300)
plt.show()

In [ ]:
# CART tuning
param_grid = {
    'max_depth': [i for i in range(1, 10)],
    'min_samples_leaf': [i for i in range(1, 6)],
    'min_samples_split': [i for i in range(2, 6)],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
}
model = DecisionTreeClassifier()
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    scoring=scoring, cv=kfold)
grid.fit(x_train_sc, y_train)
print('\n', "CART Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# CART fit training set
clf_CART = grid.best_estimator_
clf_CART.fit(x_selected, y_train)

# CART AUC in Training set
CART_Training_AUC = roc_auc_score(
    y_train, clf_CART.predict_proba(x_selected)[:, 1])
print("CART_Training_AUC:", CART_Training_AUC)
CART_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=clf_CART.predict_proba(x_selected)[:, 1], positive=1)
print("CART_Training_AUC_95%CI:", CART_Training_AUC_CI)

# CART confusion_matrix in Training set
print("CART confusion_matrix in Training set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=clf_CART.predict(x_selected)).ravel()
print("CART Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# CART Training set Sensitivity
sensitivity = tp / (tp + fn)
print("CART Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("CART Training set 95% CI (sensitivity):", ci_sensitivity)

# CART Training set Specificity
specificity = tn / (tn + fp)
print("CART Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("CART Training set 95% CI (specificity):", ci_specificity)

# CART Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("CART Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("CART Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# CART AUC in Testing set
CART_Testing_AUC = roc_auc_score(
    y_test, clf_CART.predict_proba(x_selected_test)[:, 1])
print("CART_Testing_AUC:", CART_Testing_AUC)
CART_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_CART.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("CART_Testing_AUC_95%CI:", CART_Testing_AUC_CI)

# CART confusion_matrix in Testing set
print("CART confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=clf_CART.predict(x_selected_test)).ravel()
print("CART Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# CART Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("CART Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("CART Testing set 95% CI (sensitivity):", ci_sensitivity)

# CART Testing set Specificity
specificity = tn / (tn + fp)
print("CART Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("CART Testing set 95% CI (specificity):", ci_specificity)

# CART Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("CART Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("CART Testing set 95% CI (accuracy):", ci_accuracy)

# CART test ROC

# auc_CART 95% CI_lower：
def auc_CART_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)

AUC_CART_CI_L = auc_CART_ci_lower(
    y_true=y_test, y_score=clf_CART.predict_proba(x_selected_test)[:, 1], positive=1)
# print("CART_AUC_95%CI_lower:", AUC_CART_CI_L)

# auc_CART 95% CI_upper：
def auc_CART_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)

AUC_CART_CI_U = auc_CART_ci_upper(
    y_true=y_test, y_score=clf_CART.predict_proba(x_selected_test)[:, 1], positive=1)
# print("CART_AUC_95%CI_upper:", AUC_CART_CI_U)

# ROC
y_probs = clf_CART.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % CART_Testing_AUC + '(%0.2f' % AUC_CART_CI_L + '-' + '%0.2f)' % AUC_CART_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('CART ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\CART_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# CART SHAP
explainer = shap.TreeExplainer(clf_CART)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="bar",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="CART SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'CART_shap.png')
plt.close(fig)

In [ ]:
# cart calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_CART.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='CART Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean predicted value", fontsize=12)
plt.ylabel("Fraction of positives", fontsize=12)
plt.title("CART Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\CART_cal.png', dpi=300)
plt.show()

In [ ]:
# CART_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_CART.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('CART Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\CART_DCA.png', dpi=300)
plt.show()

In [ ]:
# single model pairwise comprision with FDR

def ROC_DelongTest(clf_names, clf_methods, x_test, y_label):
    results = []

    # p value
    for n, clf1 in enumerate(clf_methods):
        for i, clf2 in enumerate(clf_methods[n + 1:], start=n + 1):
            y_predict_1 = clf1.predict_proba(x_test)[:, 1]
            y_predict_2 = clf2.predict_proba(x_test)[:, 1]
            p_value = DelongTest(y_predict_1, y_predict_2, y_label).p_value
            auc1 = roc_auc_score(y_label, y_predict_1)
            auc2 = roc_auc_score(y_label, y_predict_2)
            results.append([clf_names[n], clf_names[i], auc1, auc2, p_value])

    # FDR
    p_values = [result[-1] for result in results]
    _, corrected_p_values, _, _ = multipletests(
        p_values, alpha=0.05, method='fdr_bh')

    # p value after FDR
    for i in range(len(results)):
        results[i][-1] = corrected_p_values[i]

    return results


models = {'LR': lr, 'KNN': clf_KNN, 'SVM': clf_SVM,
          'LDA': clf_LDA, 'GNB': clf_GNB, 'CART': clf_CART}

ROC_results = ROC_DelongTest(list(models.keys()), list(
    models.values()), x_selected_test, y_test)

df = pd.DataFrame(ROC_results, columns=[
                  'Model 1', 'Model 2', 'AUC 1', 'AUC 2', 'Adjusted p-value'])
df.to_excel(r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\all_single_ROC.xlsx', index=False)

In [ ]:
# Heatmap of single model pairwise comprision with FDR

df_pvalues = pd.read_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\all_single_ROC.xlsx')
clf_names = ['LR', 'KNN', 'SVM', 'LDA', 'GNB', 'CART']
num_models = len(clf_names)
p_value_matrix = np.zeros((num_models, num_models))

for index, row in df_pvalues.iterrows():
    i = clf_names.index(row['Model 1'])
    j = clf_names.index(row['Model 2'])
    p_value_matrix[i, j] = row['Adjusted p-value']
    p_value_matrix[j, i] = row['Adjusted p-value']

fig, ax = plt.subplots()
cmap = plt.cm.YlGnBu
heatmap = ax.imshow(p_value_matrix, interpolation='nearest', cmap=cmap)

for i in range(len(clf_names)):
    for j in range(len(clf_names)):
        if i != j:
            p_val = p_value_matrix[i, j]
            if p_val < 0.001:
                display_text = "<0.001"
                color = 'red'
                weight = 'bold'
            elif p_val < 0.05:
                display_text = f"{p_val:.3f}"
                color = 'red'
                weight = 'bold'
            else:
                display_text = f"{p_val:.3f}"
                color = 'black'
                weight = 'normal'
            text = ax.text(j, i, display_text,
                           ha="center", va="center", color=color, fontweight=weight)

ax.set_xticks(np.arange(num_models))
ax.set_yticks(np.arange(num_models))
ax.set_xticklabels(clf_names, fontsize=12)
ax.set_yticklabels(clf_names, fontsize=12)
plt.xticks(rotation=45, fontsize=12)
ax.set_title(
    'Adjusted p-values from Delong Test with FDR Correction', fontsize=15)

cbar = fig.colorbar(heatmap, ax=ax, shrink=0.6, aspect=30)
cbar.ax.set_ylabel('Adjusted p-value', fontsize=12)

plt.tight_layout()
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\all_single_heatmap_FDR_corrected.png', dpi=300)
plt.show()

In [ ]:
# MRI and single models pairwise comprision with FDR

models = {'LR': lr, 'KNN': clf_KNN, 'SVM': clf_SVM, 'LDA': clf_LDA,
          'GNB': clf_GNB, 'CART': clf_CART, 'clf_MRI': clf_MRI}

aucs = {}
for name, model in models.items():
    if name == 'clf_MRI':
        predict = model.predict(x_test_step1[independent_vars])
    else:
        predict = model.predict_proba(x_selected_test)[:, 1]
    aucs[name] = roc_auc_score(y_test, predict)

p_values = []
comparisons = []
for i, (name_1, auc_1) in enumerate(aucs.items()):
    for j in range(i+1, len(aucs)):
        name_2, auc_2 = list(aucs.items())[j]
        if name_1 == 'clf_MRI':
            predict_1 = models[name_1].predict(x_test_step1[independent_vars])
        else:
            predict_1 = models[name_1].predict_proba(x_selected_test)[:, 1]
        if name_2 == 'clf_MRI':
            predict_2 = models[name_2].predict(x_test_step1[independent_vars])
        else:
            predict_2 = models[name_2].predict_proba(x_selected_test)[:, 1]
        pvalue = DelongTest(predict_1, predict_2, y_test).p_value
        p_values.append(pvalue)
        comparisons.append((name_1, name_2, auc_1, auc_2, pvalue))

rejected, p_values_corrected, _, _ = multipletests(
    p_values, alpha=0.05, method='fdr_bh')

for i in range(len(comparisons)):
    comparisons[i] = comparisons[i][:4] + (p_values_corrected[i],)

df = pd.DataFrame(comparisons, columns=[
                  'Model 1', 'Model 2', 'AUC 1', 'AUC 2', 'Adjusted p-value'])
df.to_excel('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\Single_MRI_ROC.xlsx', index=False)

In [ ]:
# Heatmap of MRI and single models pairwise comprision with FDR
df_pvalues = pd.read_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\Single_MRI_ROC.xlsx')

df_pvalues.replace('clf_MRI', 'Logit', inplace=True)
clf_names = ['Logit', 'LR', 'KNN', 'SVM', 'LDA', 'GNB', 'CART']
num_models = len(clf_names)
p_value_matrix = np.zeros((num_models, num_models))

for index, row in df_pvalues.iterrows():
    i = clf_names.index(row['Model 1'])
    j = clf_names.index(row['Model 2'])
    p_value_matrix[i, j] = row['Adjusted p-value']
    p_value_matrix[j, i] = row['Adjusted p-value']

fig, ax = plt.subplots()
cmap = plt.cm.YlGnBu
heatmap = ax.imshow(p_value_matrix, interpolation='nearest', cmap=cmap)

for i in range(len(clf_names)):
    for j in range(len(clf_names)):
        if i != j:
            p_val = p_value_matrix[i, j]
            if p_val < 0.001:
                display_text = "<0.001"
                color = 'red'
                weight = 'bold'
            elif p_val < 0.05:
                display_text = f"{p_val:.3f}"
                color = 'red'
                weight = 'bold'
            else:
                display_text = f"{p_val:.3f}"
                color = 'black'
                weight = 'normal'
            text = ax.text(j, i, display_text,
                           ha="center", va="center", color=color, fontweight=weight)

ax.set_xticks(np.arange(num_models))
ax.set_yticks(np.arange(num_models))
ax.set_xticklabels(clf_names, fontsize=12)
ax.set_yticklabels(clf_names, fontsize=12)
plt.xticks(rotation=45, fontsize=12)
ax.set_title(
    'Adjusted p-values from Delong Test with FDR Correction', fontsize=15)

cbar = fig.colorbar(heatmap, ax=ax, shrink=0.6, aspect=30)
cbar.ax.set_ylabel('Adjusted p-value', fontsize=12)

plt.tight_layout()
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\single_MRI_heatmap_FDR_corrected.png', dpi=300)
plt.show()

In [ ]:
# Bar plot of MRI and single models 

aucs = {'Logit': clf_MRI_test_auc, 'LR': LR_Testing_AUC, 'KNN': KNN_Testing_AUC, 'SVM': SVM_Testing_AUC, 'LDA': LDA_Testing_AUC, 'GNB': GNB_Testing_AUC, 'CART': CART_Testing_AUC}
conf_intervals = { 'Logit': clf_MRI_test_auc_CI,'LR': LR_Testing_AUC_CI, 'KNN': KNN_Testing_AUC_CI, 'SVM': SVM_Testing_AUC_CI, 'LDA': LDA_Testing_AUC_CI, 'GNB': GNB_Testing_AUC_CI, 'CART': CART_Testing_AUC_CI}

fig, ax = plt.subplots()
ax.bar(
    aucs.keys(),
    aucs.values(),
    color=['red', 'green', 'blue', 'orange', 'purple', 'gray', 'pink'],
    yerr=[(conf_intervals[m][1] - conf_intervals[m][0]) / 2 for m in aucs.keys()],
    capsize=5,
)
ax.set_title('AUC Comparisons', fontsize=15)
ax.set_xticklabels(aucs.keys(), rotation=45, ha='right', fontsize=12)
ax.set_ylabel('AUC', fontsize=12)
plt.title('Logit and Single Models', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Logit_single_C.png', dpi = 300)
plt.show()

In [ ]:
# NRI and IDI：single models vs MRI

y_pred_MRI = clf_MRI.predict(x_test_step1[independent_vars])
df_MRI = pd.DataFrame(y_pred_MRI, columns=['MRI_pred']).reset_index(drop=True)
y_true = y_test
y_true = y_test.reset_index(drop=True)

# LR
y_pred_lr = lr.predict_proba(x_selected_test)[:, 1]
df_lr = pd.DataFrame(y_pred_lr, columns=['lr_pred']).reset_index(drop=True)
df_MRI_lr = pd.concat([df_MRI, df_lr, y_true], axis=1)
print('NRI and IDI of MRI and LR:', '\n')
lr_nri = nri(df_MRI_lr, 'MRI_pred', 'lr_pred', 'label')
lr_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_LR.xlsx", index=False)

# KNN
y_pred_KNN = clf_KNN.predict_proba(x_selected_test)[:, 1]
y_pred_KNN = np.where(y_pred_KNN == 0, 0.000001, y_pred_KNN)
y_pred_KNN = np.where(y_pred_KNN == 1, 0.999999, y_pred_KNN)
df_KNN = pd.DataFrame(y_pred_KNN, columns=['KNN_pred']).reset_index(drop=True)
df_MRI_KNN = pd.concat([df_MRI, df_KNN, y_true], axis=1)
print('NRI and IDI of MRI and KNN:', '\n')
KNN_nri = nri(df_MRI_KNN, 'MRI_pred', 'KNN_pred', 'label')
KNN_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_KNN.xlsx", index=False)

# SVM
y_pred_SVM = clf_SVM.predict_proba(x_selected_test)[:, 1]
y_pred_SVM = np.where(y_pred_SVM == 0, 0.000001, y_pred_SVM)
y_pred_SVM = np.where(y_pred_SVM == 1, 0.999999, y_pred_SVM)
df_SVM = pd.DataFrame(y_pred_SVM, columns=['SVM_pred']).reset_index(drop=True)
df_MRI_SVM = pd.concat([df_MRI, df_SVM, y_true], axis=1)
print('NRI and IDI of MRI and SVM:', '\n')
SVM_nri = nri(df_MRI_SVM, 'MRI_pred', 'SVM_pred', 'label')
SVM_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_SVM.xlsx", index=False)

# LDA
y_pred_LDA = clf_LDA.predict_proba(x_selected_test)[:, 1]
y_pred_LDA = np.where(y_pred_LDA == 0, 0.000001, y_pred_LDA)
y_pred_LDA = np.where(y_pred_LDA == 1, 0.999999, y_pred_LDA)
df_LDA = pd.DataFrame(y_pred_LDA, columns=['LDA_pred']).reset_index(drop=True)
df_MRI_LDA = pd.concat([df_MRI, df_LDA, y_true], axis=1)
print('NRI and IDI of MRI and LDA:', '\n')
LDA_nri = nri(df_MRI_LDA, 'MRI_pred', 'LDA_pred', 'label')
LDA_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_LDA.xlsx", index=False)

# GNB
y_pred_GNB = clf_GNB.predict_proba(x_selected_test)[:, 1]
y_pred_GNB = np.where(y_pred_GNB == 0, 0.000001, y_pred_GNB)
y_pred_GNB = np.where(y_pred_GNB == 1, 0.999999, y_pred_GNB)
df_GNB = pd.DataFrame(y_pred_GNB, columns=['GNB_pred']).reset_index(drop=True)
df_MRI_GNB = pd.concat([df_MRI, df_GNB, y_true], axis=1)
print('NRI and IDI of MRI and GNB:', '\n')
GNB_nri = nri(df_MRI_GNB, 'MRI_pred', 'GNB_pred', 'label')
GNB_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_GNB.xlsx", index=False)

# CART
y_pred_CART = clf_CART.predict_proba(x_selected_test)[:, 1]
y_pred_CART = np.where(y_pred_CART == 0, 0.000001, y_pred_CART)
y_pred_CART = np.where(y_pred_CART == 1, 0.999999, y_pred_CART)
df_CART = pd.DataFrame(y_pred_CART, columns=[
                       'CART_pred']).reset_index(drop=True)
df_MRI_CART = pd.concat([df_MRI, df_CART, y_true], axis=1)
print('NRI and IDI of MRI and CART:', '\n')
CART_nri = nri(df_MRI_CART, 'MRI_pred', 'CART_pred', 'label')
CART_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_CART.xlsx", index=False)

In [ ]:
# RF tuning

param_grid = {
    'n_estimators': [20, 30, 50],
    'max_depth': [2, 5, 10, 20, 30],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}
model = RandomForestClassifier(criterion='gini')
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid,
                    scoring=scoring, cv=kfold)
grid.fit(x_train_sc, y_train)
print('\n', "RF Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# RF fit train set
clf_RF = grid.best_estimator_
clf_RF.fit(x_selected, y_train)

# RF AUC in Training set
RF_Training_AUC = roc_auc_score(
    y_train, clf_RF.predict_proba(x_selected)[:, 1])
print("RF_Training_AUC:", RF_Training_AUC)
RF_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=clf_RF.predict_proba(x_selected)[:, 1], positive=1)
print("RF_Training_AUC_95%CI:", RF_Training_AUC_CI)

# RF confusion_matrix in Training set
print("RF confusion_matrix in Training set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=clf_RF.predict(x_selected)).ravel()
print("RF Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# RF Training set Sensitivity
sensitivity = tp / (tp + fn)
print("RF Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("RF Training set 95% CI (sensitivity):", ci_sensitivity)

# RF Training set Specificity
specificity = tn / (tn + fp)
print("RF Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("RF Training set 95% CI (specificity):", ci_specificity)

# RF Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("RF Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("RF Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# RF AUC in Testing set
RF_Testing_AUC = roc_auc_score(
    y_test, clf_RF.predict_proba(x_selected_test)[:, 1])
print("RF_Testing_AUC:", RF_Testing_AUC)
RF_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_RF.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("RF_Testing_AUC_95%CI:", RF_Testing_AUC_CI)

# RF confusion_matrix in Testing set
print("RF confusion_matrix in Testing set：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=clf_RF.predict(x_selected_test)).ravel()
print("RF Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# RF Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("RF Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("RF Testing set 95% CI (sensitivity):", ci_sensitivity)

# RF Testing set Specificity
specificity = tn / (tn + fp)
print("RF Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("RF Testing set 95% CI (specificity):", ci_specificity)

# RF Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("RF Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("RF Testing set 95% CI (accuracy):", ci_accuracy)

# RF test ROC

# auc_RF 95% CI_lower：
def auc_RF_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_RF_CI_L = auc_RF_ci_lower(
    y_true=y_test, y_score=clf_RF.predict_proba(x_selected_test)[:, 1], positive=1)
# print("RF_AUC_95%CI_lower:", AUC_RF_CI_L)

# auc_RF 95% CI_upper：
def auc_RF_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_RF_CI_U = auc_RF_ci_upper(
    y_true=y_test, y_score=clf_RF.predict_proba(x_selected_test)[:, 1], positive=1)
# print("RF_AUC_95%CI_upper:", AUC_RF_CI_U)

# ROC
y_probs = clf_RF.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % RF_Testing_AUC + '(%0.2f' % AUC_RF_CI_L + '-' + '%0.2f)' % AUC_RF_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('RF ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\RF_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# RF SHAP
explainer = shap.TreeExplainer(clf_RF)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="bar",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="RF SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'RF_shap.png')
plt.close(fig)

In [ ]:
# RF calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_RF.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='RF Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("RF Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\RF_cal.png', dpi=300)
plt.show()

In [ ]:
# RF_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_RF.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('RF Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\RF_DCA.png', dpi=300)
plt.show()

In [ ]:
# Bagging tuning

param_grid = {'n_estimators': [10, 50, 100, 200, 500],
              'max_features': [1.0, 2.0, 3.0, 5.0],
              'random_state': [5, 10, 30]
              }

model = BaggingClassifier(base_estimator=clf_CART, n_jobs=1)

grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    scoring='accuracy',
                    cv=10)
grid.fit(x_train_sc, y_train)

print('\n', "bag Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# Bagging fit training set
bag_best_parameters = grid.best_estimator_
bag = grid.best_estimator_
bag.fit(x_selected, y_train)

# bag_Training_AUC
bag_Training_AUC = roc_auc_score(y_train, bag.predict_proba(x_selected)[:, 1])
print("bag_Training_AUC:", bag_Training_AUC)
bag_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=bag.predict_proba(x_selected)[:, 1], positive=1)
print("bag_Training_AUC_95%CI:", bag_Training_AUC_CI)

# bag Training set Confusion matrix
print("bag Training set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=bag.predict(x_selected)).ravel()
print("bag Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# bag Training set Sensitivity
sensitivity = tp / (tp + fn)
print("bag Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("bag Training set 95% CI (sensitivity):", ci_sensitivity)

# bag Training set Specificity
specificity = tn / (tn + fp)
print("bag Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("bag Training set 95% CI (specificity):", ci_specificity)

# bag Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("bag Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("bag Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# bag_Testing_AUC
bag_Testing_AUC = roc_auc_score(
    y_test, bag.predict_proba(x_selected_test)[:, 1])
print("bag_Testing_AUC:", bag_Testing_AUC)
bag_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=bag.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("bag_Testing_AUC_95%CI:", bag_Testing_AUC_CI)

# bag Testing set Confusion matrix
print("bag Testing set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=bag.predict(x_selected_test)).ravel()
print("bag Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# bag Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("bag Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("bag Testing set 95% CI (sensitivity):", ci_sensitivity)

# bag Testing set Specificity
specificity = tn / (tn + fp)
print("bag Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("bag Testing set 95% CI (specificity):", ci_specificity)

# bag Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("bag Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("bag Testing set 95% CI (accuracy):", ci_accuracy)

# bag test ROC

# auc_bag 95% CI_lower：
def auc_bag_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_bag_CI_L = auc_bag_ci_lower(
    y_true=y_test, y_score=bag.predict_proba(x_selected_test)[:, 1], positive=1)
# print("bag_AUC_95%CI_lower:", AUC_bag_CI_L)

# auc_bag 95% CI_upper：
def auc_bag_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC ** 2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC ** 2) +
                  (N2 - 1) * (Q2 - AUC ** 2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_bag_CI_U = auc_bag_ci_upper(
    y_true=y_test, y_score=bag.predict_proba(x_selected_test)[:, 1], positive=1)
# print("bag_AUC_95%CI_upper:", AUC_bag_CI_U)

# ROC
y_probs = bag.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr, tpr, color='darkorange', lw=1,
         label='AUC = %0.2f' % bag_Testing_AUC + '(%0.2f' % AUC_bag_CI_L + '-' + '%0.2f)' % AUC_bag_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Bagging ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Bag_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# Bagging SHAP
best_bag_model = grid.best_estimator_
shap_values_list = []

for estimator in best_bag_model.estimators_:
    explainer = shap.TreeExplainer(estimator)
    shap_values = explainer.shap_values(x_selected_test)
    if len(shap_values) == 2:
        shap_values = shap_values[1]
    shap_values_list.append(shap_values)

average_shap_values = np.mean(np.array(shap_values_list), axis=0)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(average_shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="Bag SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'Bag_shap.png')
plt.close(fig)

In [ ]:
# Bagging calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, bag.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='Bagging Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=15)
plt.ylabel("Fraction of Positives", fontsize=15)
plt.title("Bagging Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Bag_cal.png', dpi=300)
plt.show()

In [ ]:
# bagging_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = bag.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('Bagging Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Bag_DCA.png', dpi=300)
plt.show()

In [ ]:
# AdaBoost tuning
param_grid = {'learning_rate': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 10],
              'n_estimators': [10, 100, 200, 300, 400],

              }
model = AdaBoostClassifier(base_estimator=clf_CART, algorithm="SAMME.R")

grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    scoring='accuracy',
                    cv=10)
grid.fit(x_train_sc, y_train)

print('\n', "bag Best: %f using: %s" % (grid.best_score_, grid.best_params_))

# AdaBoost fit training set
ada_best_parameters = grid.best_estimator_
ada = grid.best_estimator_
ada.fit(x_selected, y_train)

# AdaBoost_Training_AUC
AdaBoost_Training_AUC = roc_auc_score(
    y_train, ada.predict_proba(x_selected)[:, 1])
print("AdaBoost_Training_AUC:", AdaBoost_Training_AUC)
AdaBoost_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=ada.predict_proba(x_selected)[:, 1], positive=1)
print("AdaBoost_Training_AUC_95%CI:", AdaBoost_Training_AUC_CI)

# AdaBoost Training set Confusion matrix
print("AdaBoost Training set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=ada.predict(x_selected)).ravel()
print("AdaBoost Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# AdaBoost Training set Sensitivity
sensitivity = tp / (tp + fn)
print("AdaBoost Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("AdaBoost Training set 95% CI (sensitivity):", ci_sensitivity)

# AdaBoost Training set Specificity
specificity = tn / (tn + fp)
print("AdaBoost Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("AdaBoost Training set 95% CI (specificity):", ci_specificity)

# AdaBoost Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("AdaBoost Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("AdaBoost Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# AdaBoost_Testing_AUC
AdaBoost_Testing_AUC = roc_auc_score(
    y_test, ada.predict_proba(x_selected_test)[:, 1])
print("AdaBoost_Testing_AUC:", AdaBoost_Testing_AUC)
AdaBoost_Testing_AUC_CI = auc_ci(
    y_true=y_test, y_score=ada.predict_proba(x_selected_test)[:, 1], positive=1)
print("AdaBoost_Testing_AUC_95%CI:", AdaBoost_Testing_AUC_CI)

# AdaBoost Testing set Confusion matrix
print("AdaBoost Testing set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=ada.predict(x_selected_test)).ravel()
print("AdaBoost Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# AdaBoost Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("AdaBoost Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("AdaBoost Testing set 95% CI (sensitivity):", ci_sensitivity)

# AdaBoost Testing set Specificity
specificity = tn / (tn + fp)
print("AdaBoost Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("AdaBoost Testing set 95% CI (specificity):", ci_specificity)

# AdaBoost Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("AdaBoost Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("AdaBoost Testing set 95% CI (accuracy):", ci_accuracy)

# AdaBoost test set ROC

# auc_ada 95% CI_lower：
def auc_ada_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_ada_CI_L = auc_ada_ci_lower(y_true=y_test,
                                y_score=ada.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("ada_AUC_95%CI_lower:", AUC_ada_CI_L)

# auc_ada 95% CI_upper：
def auc_ada_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_ada_CI_U = auc_ada_ci_upper(y_true=y_test,
                                y_score=ada.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("ada_AUC_95%CI_upper:", AUC_ada_CI_U)

# ROC
y_probs = ada.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr,
         tpr,
         color='darkorange',
         lw=1,
         label='AUC = %0.2f' % AdaBoost_Testing_AUC + '(%0.2f' % AUC_ada_CI_L + '-' +
         '%0.2f)' % AUC_ada_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('AdaBoost ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\AdaBoost_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# AdaBoost SHAP
best_bag_model = grid.best_estimator_
shap_values_list = []

for estimator in best_bag_model.estimators_:
    explainer = shap.TreeExplainer(estimator)
    shap_values = explainer.shap_values(x_selected_test)
    if len(shap_values) == 2:
        shap_values = shap_values[1]
    shap_values_list.append(shap_values)

average_shap_values = np.mean(np.array(shap_values_list), axis=0)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(average_shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="AdaBoost SHAP Value")
plt.yticks(fontsize=4)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'AdaBoost_shap.png')
plt.close(fig)

In [ ]:
# AdaBoost calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, ada.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='AdaBoost Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("AdaBoost Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\AdaBoost_cal.png', dpi=300)
plt.show()

In [ ]:
# Adaboost_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = ada.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('AdaBoost Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\AdaBoost_DCA.png', dpi=300)
plt.show()

In [ ]:
# GBDT parameters：The parameters 'n_estimators' and 'learning_rate' are set using the tuned parameters from AdaBoost, while the remaining parameters are configured using the tuned parameters from CART and RF.

clf_GBDT = GBDT(loss='deviance',
                n_estimators=400,
                learning_rate=1,
                min_samples_split=8,
                min_samples_leaf=2,
                max_depth=4,
                max_features='sqrt',
                subsample=0.8,
                random_state=None)

# GBDT fit training set
clf_GBDT.fit(x_selected, y_train)

# GBDT_Training_AUC
GBDT_Training_AUC = roc_auc_score(
    y_train, clf_GBDT.predict_proba(x_selected)[:, 1])
print("GBDT_Training_AUC:", GBDT_Training_AUC)
GBDT_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=clf_GBDT.predict_proba(x_selected)[:, 1], positive=1)
print("GBDT_Training_AUC_95%CI:", GBDT_Training_AUC_CI)

# GBDT Training set Confusion matrix
print("GBDT Training set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=clf_GBDT.predict(x_selected)).ravel()
print("GBDT Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# GBDT Training set Sensitivity
sensitivity = tp / (tp + fn)
print("GBDT Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("GBDT Training set 95% CI (sensitivity):", ci_sensitivity)

# GBDT Training set Specificity
specificity = tn / (tn + fp)
print("GBDT Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("GBDT Training set 95% CI (specificity):", ci_specificity)

# GBDT Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("GBDT Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("GBDT Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# GBDT_Testing_AUC
GBDT_Testing_AUC = roc_auc_score(
    y_test, clf_GBDT.predict_proba(x_selected_test)[:, 1])
print("GBDT_Testing_AUC:", GBDT_Testing_AUC)
GBDT_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=clf_GBDT.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("GBDT_Testing_AUC_95%CI:", GBDT_Testing_AUC_CI)

# GBDT Testing set Confusion matrix
print("GBDT Testing set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=clf_GBDT.predict(x_selected_test)).ravel()
print("GBDT Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# GBDT Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("GBDT Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("GBDT Testing set 95% CI (sensitivity):", ci_sensitivity)

# GBDT Testing set Specificity
specificity = tn / (tn + fp)
print("GBDT Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("GBDT Testing set 95% CI (specificity):", ci_specificity)

# GBDT Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("GBDT Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("GBDT Testing set 95% CI (accuracy):", ci_accuracy)

# GBDT test ROC

# auc_GBDT 95% CI_lower：
def auc_GBDT_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_GBDT_CI_L = auc_GBDT_ci_lower(
    y_true=y_test,
    y_score=clf_GBDT.predict_proba(x_selected_test)[:, 1],
    positive=1)
# print("GBDT_AUC_95%CI_lower:", AUC_GBDT_CI_L)

# auc_GBDT 95% CI_upper：
def auc_GBDT_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_GBDT_CI_U = auc_GBDT_ci_upper(
    y_true=y_test,
    y_score=clf_GBDT.predict_proba(x_selected_test)[:, 1],
    positive=1)
# print("GBDT_AUC_95%CI_upper:", AUC_GBDT_CI_U)

# ROC
y_probs = clf_GBDT.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr,
         tpr,
         color='darkorange',
         lw=1,
         label='AUC = %0.2f' % GBDT_Testing_AUC + '(%0.2f' % AUC_GBDT_CI_L + '-' +
         '%0.2f)' % AUC_GBDT_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('GBDT ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\GBDT_ROC.png', dpi=300)
plt.show()

winsound.Beep(frequency, duration)

In [ ]:
# GBDT SHAP
explainer = shap.TreeExplainer(clf_GBDT)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="GBDT SHAP Value")

plt.yticks(fontsize=4)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'GBDT_shap.png')
plt.close(fig)

In [ ]:
# GBDT calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, clf_GBDT.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='GBDT Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("GBDT Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\GBDT_cal.png', dpi=300)
plt.show()

In [ ]:
# GBDT_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = clf_GBDT.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('GBDT Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\GBDT_DCA.png', dpi=300)
plt.show()

In [ ]:
# XGBoost tuning with shap-hypetune

param_dist_hyperopt = {
    'n_estimators':hp.randint('n_estimators',500),
    'max_depth': 15 + hp.randint('num_leaves', 5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}
clf_xgb = XGBClassifier(random_state=0, verbosity=0, n_jobs=-1)
model = BoostSearch(clf_xgb,
                    param_grid=param_dist_hyperopt,
                    n_iter=8,
                    sampling_seed=0)
model.fit(x_train_sc, y_train,
          trials=Trials(),
          eval_set=[(x_selected, y_train)],
          early_stopping_rounds=6,
          verbose=0)
model.estimator_, model.best_params_, model.best_score_
print('\n','shap-hypetune xgb：', model.best_params_)

In [ ]:
# The XGB model employs the tuned parameters derived from the aforementioned SHAP-hypertune process.

XGB = XGBClassifier(n_estimators=279,
                    learning_rate=0.1,
                    max_depth=16,
                    min_child_weight=1,
                    gamma=0.0,
                    subsample=0.6,
                    colsample_bytree=0.8,
                    objective='binary:logistic',
                    scale_pos_weight=1,
                    seed=27)
XGB = XGB.fit(x_selected, y_train)

# XGB_Training_AUC
XGB_Training_AUC = roc_auc_score(y_train, XGB.predict_proba(x_selected)[:, 1])
print("XGB_Training_AUC:", XGB_Training_AUC)
XGB_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=XGB.predict_proba(x_selected)[:, 1], positive=1)
print("XGB_Training_AUC_95%CI:", XGB_Training_AUC_CI)

# XGB Training set Confusion matrix
print("XGB Training set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=XGB.predict(x_selected)).ravel()
print("XGB Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# XGB Training set Sensitivity
sensitivity = tp / (tp + fn)
print("XGB Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("XGB Training set 95% CI (sensitivity):", ci_sensitivity)

# XGB Training set Specificity
specificity = tn / (tn + fp)
print("XGB Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("XGB Training set 95% CI (specificity):", ci_specificity)

# XGB Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("XGB Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("XGB Training set 95% CI (accuracy):", ci_accuracy)
print(" ")

# XGB_Testing_AUC
XGB_Testing_AUC = roc_auc_score(
    y_test, XGB.predict_proba(x_selected_test)[:, 1])
print("XGB_Testing_AUC:", XGB_Testing_AUC)
XGB_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=XGB.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("XGB_Testing_AUC_95%CI:", XGB_Testing_AUC_CI)

# XGB Testing set Confusion matrix
print("XGB Testing set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=XGB.predict(x_selected_test)).ravel()
print("XGB Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# XGB Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("XGB Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("XGB Testing set 95% CI (sensitivity):", ci_sensitivity)

# XGB Testing set Specificity
specificity = tn / (tn + fp)
print("XGB Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("XGB Testing set 95% CI (specificity):", ci_specificity)

# XGB Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("XGB Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("XGB Testing set 95% CI (accuracy):", ci_accuracy)

# XGB test set ROC

# auc_XGB 95% CI_lower：
def auc_XGB_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_XGB_CI_L = auc_XGB_ci_lower(y_true=y_test,
                                y_score=XGB.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("调参后XGB_AUC_95%CI_lower:", AUC_XGB_CI_L)


# auc_XGB 95% CI_upper：
def auc_XGB_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_XGB_CI_U = auc_XGB_ci_upper(y_true=y_test,
                                y_score=XGB.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("调参后XGB_AUC_95%CI_upper:", AUC_XGB_CI_U)

# ROC
y_probs = XGB.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr,
         tpr,
         color='darkorange',
         lw=1,
         label='AUC = %0.2f' % XGB_Testing_AUC + '(%0.2f' % AUC_XGB_CI_L + '-' +
         '%0.2f)' % AUC_XGB_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('XGBoost ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\XGBoost_ROC.png', dpi=300)
plt.show()

In [ ]:
# XGB SHAP
explainer = shap.TreeExplainer(XGB)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="XGB SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'XGB_shap.png')
plt.close(fig)

In [ ]:
# XGBoost calibration_curve
fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, XGB.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='XGBoost Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("XGBoost Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\XGBoost_cal.png', dpi=300)
plt.show()

In [ ]:
# XGBoost_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = XGB.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('XGBoost Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\XGBoost_DCA.png', dpi=300)
plt.show()

In [ ]:
# LightGBM tuning with shap-hypetune

param_dist_hyperopt = {
    'n_estimators': hp.randint('n_estimators', 500),
    'max_depth': 15 + hp.randint('num_leaves', 5),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

clf_lgbm = LGBMClassifier(random_state=0, n_jobs=-1)

model = BoostSearch(clf_lgbm, param_grid=param_dist_hyperopt,
                    n_iter=8, sampling_seed=0)
model.fit(x_train_sc, y_train, trials=Trials(), eval_set=[
          (x_selected, y_train)], early_stopping_rounds=6, verbose=0)
model.estimator_, model.best_params_, model.best_score_
print('\n', 'shap-hypetune ightGBM', model.best_params_)

In [ ]:
# The LightGBM model employs the tuned parameters derived from the aforementioned SHAP-hypertune process.

lgb = LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=19,
    learning_rate=0.011,
    n_estimators=357,
    subsample_for_bin=200000,
    objective=None,
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=0.74,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None,
    n_jobs=-1,
    silent=True,)
lgb = lgb.fit(x_selected, y_train)

# lgb_Training_AUC
lgb_Training_AUC = roc_auc_score(y_train, lgb.predict_proba(x_selected)[:, 1])
print("lgb_Training_AUC:", lgb_Training_AUC)
lgb_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=lgb.predict_proba(x_selected)[:, 1], positive=1)
print("lgb_Training_AUC_95%CI:", lgb_Training_AUC_CI)

# lgb Training set Confusion matrix
print("lgb Training set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=lgb.predict(x_selected)).ravel()
print("lgb Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# lgb Training set Sensitivity
sensitivity = tp / (tp + fn)
print("lgb Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("lgb Training set 95% CI (sensitivity):", ci_sensitivity)

# lgb Training set Specificity
specificity = tn / (tn + fp)
print("lgb Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("lgb Training set 95% CI (specificity):", ci_specificity)

# lgb Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("lgb Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("lgb Training set 95% CI (accuracy):", ci_accuracy)

print(" ")

# lgb_Testing_AUC
lgb_Testing_AUC = roc_auc_score(
    y_test, lgb.predict_proba(x_selected_test)[:, 1])
print("lgb_Testing_AUC:", lgb_Testing_AUC)
lgb_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=lgb.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("lgb_Testing_AUC_95%CI:", lgb_Testing_AUC_CI)

# lgb Testing set Confusion matrix
print("lgb Testing set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=lgb.predict(x_selected_test)).ravel()
print("lgb Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# lgb Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("lgb Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("lgb Testing set 95% CI (sensitivity):", ci_sensitivity)

# lgb Testing set Specificity
specificity = tn / (tn + fp)
print("lgb Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("lgb Testing set 95% CI (specificity):", ci_specificity)

# lgb Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("lgb Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("lgb Testing set 95% CI (accuracy):", ci_accuracy)

# lgb Testing set ROC

# auc_lgb 95% CI_lower：
def auc_lgb_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_lgb_CI_L = auc_lgb_ci_lower(y_true=y_test,
                                y_score=lgb.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("调参后lgb_AUC_95%CI_lower:", AUC_lgb_CI_L)

# auc_lgb 95% CI_upper：
def auc_lgb_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_lgb_CI_U = auc_lgb_ci_upper(y_true=y_test,
                                y_score=lgb.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("调参后lgb_AUC_95%CI_upper:", AUC_lgb_CI_U)

# ROC
y_probs = lgb.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr,
         tpr,
         color='darkorange',
         lw=1,
         label='AUC = %0.2f' % lgb_Testing_AUC + '(%0.2f' % AUC_lgb_CI_L + '-' +
         '%0.2f)' % AUC_lgb_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('LightGBM ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LightGBM_ROC.png', dpi=300)
plt.show()

In [ ]:
# LightGBM SHAP
explainer = shap.TreeExplainer(lgb)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="bar",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="LGB SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'LBG_shap.png')
plt.close(fig)

In [ ]:
# LightGBM calibration_curve

fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, lgb.predict_proba(x_selected_test)[:,1], n_bins = 4
    #                   , strategy = 'quantile'
                     )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives, "rs-", label='LightGBM Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("LightGBM Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LightGBM_cal.png', dpi = 300)
plt.show()

In [ ]:
# lightGBM_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = lgb.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('LightGBM Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\LightGBM_DCA.png', dpi=300)
plt.show()

In [ ]:
# CatBoost tuning with CatboostTuner
import catboost as Catboost
import CatboostTuner

tuned_catboost = CatboostTuner.CatboostTuner()
tuned_catboost.run(X=x_train_sc, y=y_train, nfold=5)

predictions = tuned_catboost.predict(x_test_sc)
print("Test MSE:", mean_squared_error(y_test, predictions))
print(tuned_catboost.params)

In [ ]:
#  The Catboost model employs the tuned parameters derived from the aforementioned CatboostTuner process.

from catboost import CatBoostClassifier

cat = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    iterations=1100,
    learning_rate=0.03,
    bootstrap_type='Bernoulli',
    subsample=0.4,
    random_strength=1,
    rsm=0.1,
    max_depth=3,
    grow_policy='SymmetricTree',
    l2_leaf_reg=1,
)
cat = cat.fit(x_selected, y_train)

# cat_Training_AUC
cat_Training_AUC = roc_auc_score(y_train, cat.predict_proba(x_selected)[:, 1])
print("cat_Training_AUC:", cat_Training_AUC)
cat_Training_AUC_CI = auc_ci(
    y_true=y_train, y_score=cat.predict_proba(x_selected)[:, 1], positive=1)
print("cat_Training_AUC_95%CI:", cat_Training_AUC_CI)

# cat Training set Confusion matrix
print("cat Training set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_train, y_pred=cat.predict(x_selected)).ravel()
print("cat Training set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))


# cat Training set Sensitivity
sensitivity = tp / (tp + fn)
print("cat Training set Sensitivity:", sensitivity)
n_samples_sen = fn + tp
n_samples = n_samples_sen
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("cat Training set 95% CI (sensitivity):", ci_sensitivity)

# cat Training set Specificity
specificity = tn / (tn + fp)
print("cat Training set Specificity:", specificity)
n_samples_spe = tn + fp
n_samples = n_samples_spe
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("cat Training set 95% CI (specificity):", ci_specificity)

# cat Training set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("cat Training set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("cat Training set 95% CI (accuracy):", ci_accuracy)

print(" ")

# cat_Testing_AUC
cat_Testing_AUC = roc_auc_score(
    y_test, cat.predict_proba(x_selected_test)[:, 1])
print("cat_Testing_AUC:", cat_Testing_AUC)
cat_Testing_AUC_CI = auc_ci(y_true=y_test, y_score=cat.predict_proba(
    x_selected_test)[:, 1], positive=1)
print("cat_Testing_AUC_95%CI:", cat_Testing_AUC_CI)

# cat Testing set Confusion matrix
print("cat Testing set Confusion matrix：")
tn, fp, fn, tp = confusion_matrix(
    y_true=y_test, y_pred=cat.predict(x_selected_test)).ravel()
print("cat Testing set Confusion matrix:")
print("TN={}, FP={}".format(tn, fp))
print("FN={}, TP={}".format(fn, tp))

# cat Testing set Sensitivity
sensitivity = tp / (tp + fn)
print("cat Testing set Sensitivity:", sensitivity)
n_samples = fn + tp
ci_sensitivity = sm.proportion_confint(tp, n_samples, alpha=0.05)
print("cat Testing set 95% CI (sensitivity):", ci_sensitivity)

# cat Testing set Specificity
specificity = tn / (tn + fp)
print("cat Testing set Specificity:", specificity)
n_samples = tn + fp
ci_specificity = sm.proportion_confint(tn, n_samples, alpha=0.05)
print("cat Testing set 95% CI (specificity):", ci_specificity)

# cat Testing set accuracy
accuracy = (tn + tp) / (tn + fp + fn + tp)
print("cat Testing set accuracy:", accuracy)
n_correct = tn + tp
n_samples = tn + fp + fn + tp
ci_accuracy = sm.proportion_confint(n_correct, n_samples, alpha=0.05)
print("cat Testing set 95% CI (accuracy):", ci_accuracy)

# CatBoost test ROC curve

# auc_cat 95% CI_lower：
def auc_cat_ci_lower(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (lower)


AUC_cat_CI_L = auc_cat_ci_lower(y_true=y_test,
                                y_score=cat.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("cat_AUC_95%CI_lower:", AUC_cat_CI_L)

# auc_cat 95% CI_upper：
def auc_cat_ci_upper(y_true, y_score, positive=1):
    AUC = roc_auc_score(y_true, y_score)
    N1 = sum(y_true == positive)
    N2 = sum(y_true != positive)
    Q1 = AUC / (2 - AUC)
    Q2 = 2 * AUC**2 / (1 + AUC)
    SE_AUC = sqrt((AUC * (1 - AUC) + (N1 - 1) * (Q1 - AUC**2) + (N2 - 1) *
                   (Q2 - AUC**2)) / (N1 * N2))
    lower = AUC - 1.96 * SE_AUC
    upper = AUC + 1.96 * SE_AUC
    if lower < 0:
        lower = 0
    if upper > 1:
        upper = 1
    return (upper)


AUC_cat_CI_U = auc_cat_ci_upper(y_true=y_test,
                                y_score=cat.predict_proba(x_selected_test)[:,
                                                                           1],
                                positive=1)
# print("cat_AUC_95%CI_upper:", AUC_cat_CI_U)

# ROC
y_probs = cat.predict_proba(x_selected_test)
fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, 1], pos_label=1)
plt.figure(dpi=300)
plt.plot(fpr,
         tpr,
         color='darkorange',
         lw=1,
         label='AUC = %0.2f' % cat_Testing_AUC + '(%0.2f' % AUC_cat_CI_L + '-' +
         '%0.2f)' % AUC_cat_CI_U)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.axis('auto')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('CatBoost ROC Curve', fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\CatBoost_ROC.png', dpi=300)
plt.show()

In [ ]:
# CatBoost SHAP
explainer = shap.TreeExplainer(cat)
shap_values = explainer.shap_values(x_selected_test)

save_path = 'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\SHAP\\'
fig, ax = plt.subplots(dpi=300)
shap.summary_plot(shap_values, x_selected_test, feature_names=x_selected_test.columns, plot_type="dot",
                  show=False, plot_size=None, color_bar=True, axis_color="#333333", title="cat SHAP Value")

plt.yticks(fontsize=5)
plt.tight_layout()
plt.show()
fig.savefig(save_path + 'cat_shap.png')
plt.close(fig)

In [ ]:
# CatBoost calibration_curve

fraction_of_positives, mean_predicted_value = \
    calibration_curve(y_test, cat.predict_proba(x_selected_test)[:, 1], n_bins=4
                      #                   , strategy = 'quantile'
                      )
print(fraction_of_positives)
print(mean_predicted_value)

plt.plot(mean_predicted_value, fraction_of_positives,
         "rs-", label='CatBoost Classifier')
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel("Mean Predicted Value", fontsize=12)
plt.ylabel("Fraction of Positives", fontsize=12)
plt.title("CatBoost Calibration Curve", fontsize=15)
plt.legend(loc="lower right")
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\CatBoost_cal.png', dpi=300)
plt.show()

In [ ]:
# CatBoost_DCA
from DCA import net_benefit_model, net_benefit_all, plot_DCA

y_pred_score = cat.predict_proba(x_selected_test)[:, 1]
y_label = y_test
thresh_group = np.arange(0, 1, 0.01)
net_benefit_model = net_benefit_model(thresh_group, y_pred_score, y_label)
net_benefit_all = net_benefit_all(thresh_group, y_label)
fig, ax = plt.subplots()
ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
plt.title('CatBoost Decision Curve Analysis', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\CatBoost_DCA.png', dpi=300)
plt.show()

In [ ]:
# pairwise comprision in ensemble models with FDR
def ROC_DelongTest(clf_names, clf_methods, x_test, y_label):
    len_names = len(clf_names)
    len_clfs = len(clf_methods)
    results = []
    for n, p in zip(range(0, len_names), range(0, len_clfs)):
        for i, j in zip(range(n + 1, len_names), range(p + 1, len_clfs)):
            y_predict_1 = clf_methods[p].predict_proba(x_test)[:, 1]
            y_predict_2 = clf_methods[j].predict_proba(x_test)[:, 1]
            print('ROC comparison:', clf_names[n] + ' vs. ' + clf_names[i])
            dt = DelongTest(y_predict_1, y_predict_2, y_label)
            auc1 = roc_auc_score(y_label, y_predict_1)
            auc2 = roc_auc_score(y_label, y_predict_2)
            results.append(
                [clf_names[n], clf_names[i], auc1, auc2, dt.p_value])
            print('\n')

    p_values = [result[-1] for result in results]
    _, corrected_p_values, _, _ = multipletests(
        p_values, alpha=0.05, method='fdr_bh')

    for i in range(len(results)):
        results[i][-1] = corrected_p_values[i]

    df = pd.DataFrame(results, columns=[
                      'Model 1', 'Model 2', 'AUC 1', 'AUC 2', 'Adjusted p-value'])
    df.to_excel(
        r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\\All_em_ROC.xlsx', index=False)


ROCs = ROC_DelongTest(clf_names=[
    'RF', 'Bagging', 'AdaBoost', 'GBDT', 'XGBoost', 'LightGBM', 'CatBoost'
],
    clf_methods=[clf_RF, bag, ada, clf_GBDT, XGB, lgb, cat],
    x_test=x_selected_test,
    y_label=y_test)

In [ ]:
# Heatmap of pairwise comprision in ensemble models with FDR

df_pvalues = pd.read_excel(
    r'E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\\All_em_ROC.xlsx')

clf_names = ['RF', 'Bagging', 'AdaBoost',
             'GBDT', 'XGBoost', 'LightGBM', 'CatBoost']
num_models = len(clf_names)
p_value_matrix = np.zeros((num_models, num_models))

for index, row in df_pvalues.iterrows():
    i = clf_names.index(row['Model 1'])
    j = clf_names.index(row['Model 2'])
    p_value_matrix[i, j] = row['Adjusted p-value']
    p_value_matrix[j, i] = row['Adjusted p-value']

fig, ax = plt.subplots()
cmap = plt.cm.YlGnBu
heatmap = ax.imshow(p_value_matrix, interpolation='nearest', cmap=cmap)

for i in range(len(clf_names)):
    for j in range(len(clf_names)):
        if i != j:
            p_val = p_value_matrix[i, j]
            if p_val < 0.001:
                display_text = "<0.001"
                color = 'red'
                weight = 'bold'
            elif p_val < 0.05:
                display_text = f"{p_val:.3f}"
                color = 'red'
                weight = 'bold'
            else:
                display_text = f"{p_val:.3f}"
                color = 'black'
                weight = 'normal'
            text = ax.text(j, i, display_text,
                           ha="center", va="center", color=color, fontweight=weight)

ax.set_xticks(np.arange(num_models))
ax.set_yticks(np.arange(num_models))
ax.set_xticklabels(clf_names, fontsize=12)
ax.set_yticklabels(clf_names, fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=12)
ax.set_title(
    'Adjusted p-values from Delong Test with FDR Correction', fontsize=15)

cbar = fig.colorbar(heatmap, ax=ax, shrink=0.56, aspect=30)
cbar.ax.set_ylabel('Adjusted p-value', fontsize=12)

plt.tight_layout()
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\all_ensemble_heatmap_FDR_corrected.png', dpi=300)
plt.show()

In [ ]:
# pairwise comprision in ensemble and MRI models with FDR

models = {'Logit': clf_MRI, 'RF': clf_RF, 'Bagging': bag, 'AdaBoost': ada,
          'GBDT': clf_GBDT, 'XGBoost': XGB, 'LightGBM': lgb, 'CatBoost': cat}

aucs = {}
for name, model in models.items():
    if name == 'Logit':
        predict = model.predict(x_test_step1[independent_vars])
    else:
        predict = model.predict_proba(x_selected_test)[:, 1]
    aucs[name] = roc_auc_score(y_test, predict)

p_values = []
comparisons = []
for i, (name_1, auc_1) in enumerate(aucs.items()):
    for j in range(i+1, len(aucs)):
        name_2, auc_2 = list(aucs.items())[j]
        if name_1 == 'Logit':
            predict_1 = models[name_1].predict(x_test_step1[independent_vars])
        else:
            predict_1 = models[name_1].predict_proba(x_selected_test)[:, 1]
        if name_2 == 'Logit':
            predict_2 = models[name_2].predict(x_test_step1[independent_vars])
        else:
            predict_2 = models[name_2].predict_proba(x_selected_test)[:, 1]
        pvalue = DelongTest(predict_1, predict_2, y_test).p_value
        p_values.append(pvalue)
        comparisons.append((name_1, name_2, auc_1, auc_2, pvalue))

rejected, p_values_corrected, _, _ = multipletests(
    p_values, alpha=0.05, method='fdr_bh')

for i in range(len(comparisons)):
    comparisons[i] = comparisons[i][:4] + (p_values_corrected[i],)

df = pd.DataFrame(comparisons, columns=[
                  'Model 1', 'Model 2', 'AUC 1', 'AUC 2', 'Adjusted p-value'])
df.to_excel('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\em_MRI_ROC.xlsx', index=False)

In [ ]:
# Heatmap of pairwise comprision in ensemble and MRI models with FDR

df_pvalues = pd.read_excel(
    r'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\em_MRI_ROC.xlsx')

clf_names = ['Logit', 'RF', 'Bagging', 'AdaBoost',
             'GBDT', 'XGBoost', 'LightGBM', 'CatBoost']
num_models = len(clf_names)
p_value_matrix = np.zeros((num_models, num_models))

for index, row in df_pvalues.iterrows():
    i = clf_names.index(row['Model 1'])
    j = clf_names.index(row['Model 2'])
    p_value_matrix[i, j] = row['Adjusted p-value']
    p_value_matrix[j, i] = row['Adjusted p-value']

fig, ax = plt.subplots()
cmap = plt.cm.YlGnBu
heatmap = ax.imshow(p_value_matrix, interpolation='nearest', cmap=cmap)

for i in range(len(clf_names)):
    for j in range(len(clf_names)):
        if i != j:
            p_val = p_value_matrix[i, j]
            if p_val < 0.001:
                display_text = "<0.001"
                color = 'red'
                weight = 'bold'
            elif p_val < 0.05:
                display_text = f"{p_val:.3f}"
                color = 'red'
                weight = 'bold'
            else:
                display_text = f"{p_val:.3f}"
                color = 'black'
                weight = 'normal'
            text = ax.text(j, i, display_text,
                           ha="center", va="center", color=color, fontweight=weight)

ax.set_xticks(np.arange(num_models))
ax.set_yticks(np.arange(num_models))
ax.set_xticklabels(clf_names, fontsize=12)
ax.set_yticklabels(clf_names, fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=12)
ax.set_title(
    'Adjusted p-values from Delong Test with FDR Correction', fontsize=15)

cbar = fig.colorbar(heatmap, ax=ax, shrink=0.56, aspect=30)
cbar.ax.set_ylabel('Adjusted p-value', fontsize=12)

plt.tight_layout()
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\em_MRI_heatmap_FDR_corrected.png', dpi=300)
plt.show()

In [ ]:
# Bar plot of ensemble models and MRI model

aucs = {'RF':RF_Testing_AUC, 'Bagging':bag_Testing_AUC, 'AdaBoost':AdaBoost_Testing_AUC, 'GBDT':GBDT_Testing_AUC, 'XGBoost':XGB_Testing_AUC, 'LightGBM':lgb_Testing_AUC, 'CatBoost':cat_Testing_AUC, 'Logit': clf_MRI_test_auc}
conf_intervals = {'RF':RF_Testing_AUC_CI, 'Bagging':bag_Testing_AUC_CI, 'AdaBoost':AdaBoost_Testing_AUC_CI, 'GBDT':GBDT_Testing_AUC_CI, 'XGBoost':XGB_Testing_AUC_CI, 'LightGBM':lgb_Testing_AUC_CI, 'CatBoost':cat_Testing_AUC_CI, 'Logit': clf_MRI_test_auc_CI}

fig, ax = plt.subplots()
ax.bar(
    aucs.keys(),
    aucs.values(),
    color=['#9FE080', 'red', 'green', 'blue', 'orange', 'purple', 'gray', 'pink'],
    yerr=[(conf_intervals[m][1] - conf_intervals[m][0]) / 2 for m in aucs.keys()],
    capsize=5,
)

ax.set_title('MRI em AUC Comparisons', fontsize=16)
ax.set_xticklabels(aucs.keys(), rotation=45, ha='right', fontsize=12)
ax.set_ylabel('AUC', fontsize=12)

plt.title('Logit and Ensemble Models', fontsize=15)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Logit_em_C.png', dpi = 300)
plt.show()

In [ ]:
# NRI and IDI ensemble models VS MRI model

models = {'RF': clf_RF, 'Bagging': bag, 'AdaBoost': ada, 'GBDT': clf_GBDT,
          'XGBoost': XGB, 'LightGBM': lgb, 'CatBoost': cat, 'clf_MRI': clf_MRI}

y_pred_MRI = clf_MRI.predict(x_test_step1[independent_vars])
df_MRI = pd.DataFrame(y_pred_MRI, columns=['MRI_pred']).reset_index(drop=True)
y_true = y_test
y_true = y_test.reset_index(drop=True)

# RF
y_pred_RF = clf_RF.predict_proba(x_selected_test)[:, 1]
y_pred_RF = np.where(y_pred_RF == 0, 0.000001, y_pred_RF)
y_pred_RF = np.where(y_pred_RF == 1, 0.999999, y_pred_RF)
df_RF = pd.DataFrame(y_pred_RF, columns=['RF_pred']).reset_index(drop=True)
df_MRI_RF = pd.concat([df_MRI, df_RF, y_true], axis=1)
print('NRI and IDI of MRI and RF:', '\n')
RF_nri = nri(df_MRI_RF, 'MRI_pred', 'RF_pred', 'label')
print('\n')
RF_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_RF.xlsx")

# bagging
y_pred_bagging = bag.predict_proba(x_selected_test)[:, 1]
y_pred_bagging = np.where(y_pred_bagging == 0, 0.000001, y_pred_bagging)
y_pred_bagging = np.where(y_pred_bagging == 1, 0.999999, y_pred_bagging)
df_bagging = pd.DataFrame(y_pred_bagging, columns=[
                          'bagging_pred']).reset_index(drop=True)
df_MRI_bagging = pd.concat([df_MRI, df_bagging, y_true], axis=1)
print('NRI and IDI of MRI and bagging:', '\n')
Bagging_nri = nri(df_MRI_bagging, 'MRI_pred', 'bagging_pred', 'label')
print('\n')
Bagging_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_bagging.xlsx")

# AdaBoost
y_pred_AdaBoost = ada.predict_proba(x_selected_test)[:, 1]
y_pred_AdaBoost = np.where(y_pred_AdaBoost == 0, 0.000001, y_pred_AdaBoost)
y_pred_AdaBoost = np.where(y_pred_AdaBoost == 1, 0.999999, y_pred_AdaBoost)
df_AdaBoost = pd.DataFrame(y_pred_AdaBoost, columns=[
                           'AdaBoost_pred']).reset_index(drop=True)
df_MRI_AdaBoost = pd.concat([df_MRI, df_AdaBoost, y_true], axis=1)
print('NRI and IDI of MRI and AdaBoost:', '\n')
AdaBoost_nri = nri(df_MRI_AdaBoost, 'MRI_pred', 'AdaBoost_pred', 'label')
print('\n')
AdaBoost_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_AdaBoost.xlsx")

# GBDT
y_pred_GBDT = clf_GBDT.predict_proba(x_selected_test)[:, 1]
y_pred_GBDT = np.where(y_pred_GBDT == 0, 0.000001, y_pred_GBDT)
y_pred_GBDT = np.where(y_pred_GBDT == 1, 0.999999, y_pred_GBDT)
df_GBDT = pd.DataFrame(y_pred_GBDT, columns=[
                       'GBDT_pred']).reset_index(drop=True)
df_MRI_GBDT = pd.concat([df_MRI, df_GBDT, y_true], axis=1)
print('NRI and IDI of MRI and GBDT:', '\n')
GBDT_nri = nri(df_MRI_GBDT, 'MRI_pred', 'GBDT_pred', 'label')
print('\n')
GBDT_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_GBDT.xlsx")

# XGBoost
y_pred_XGBoost = XGB.predict_proba(x_selected_test)[:, 1]
y_pred_XGBoost = np.where(y_pred_XGBoost == 0, 0.000001, y_pred_XGBoost)
y_pred_XGBoost = np.where(y_pred_XGBoost == 1, 0.999999, y_pred_XGBoost)
df_XGBoost = pd.DataFrame(y_pred_XGBoost, columns=[
                          'XGBoost_pred']).reset_index(drop=True)
df_MRI_XGBoost = pd.concat([df_MRI, df_XGBoost, y_true], axis=1)
print('NRI and IDI of MRI and XGBoost:', '\n')
XGBoost_nri = nri(df_MRI_XGBoost, 'MRI_pred', 'XGBoost_pred', 'label')
print('\n')
XGBoost_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_XGBoost.xlsx")

# LightGBM
y_pred_LightGBM = lgb.predict_proba(x_selected_test)[:, 1]
y_pred_LightGBM = np.where(y_pred_LightGBM == 0, 0.000001, y_pred_LightGBM)
y_pred_LightGBM = np.where(y_pred_LightGBM == 1, 0.999999, y_pred_LightGBM)
df_LightGBM = pd.DataFrame(y_pred_LightGBM, columns=[
                           'LightGBM_pred']).reset_index(drop=True)
df_MRI_LightGBM = pd.concat([df_MRI, df_LightGBM, y_true], axis=1)
print('NRI and IDI of MRI and LightGBM:', '\n')
LightGBM_nri = nri(df_MRI_LightGBM, 'MRI_pred', 'LightGBM_pred', 'label')
print('\n')
LightGBM_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_LightGBM.xlsx")

# CatBoost
y_pred_CatBoost = cat.predict_proba(x_selected_test)[:, 1]
y_pred_CatBoost = np.where(y_pred_CatBoost == 0, 0.000001, y_pred_CatBoost)
y_pred_CatBoost = np.where(y_pred_CatBoost == 1, 0.999999, y_pred_CatBoost)
df_CatBoost = pd.DataFrame(y_pred_CatBoost, columns=[
                           'CatBoost_pred']).reset_index(drop=True)
df_MRI_CatBoost = pd.concat([df_MRI, df_CatBoost, y_true], axis=1)
print('NRI and IDI of MRI and CatBoost:', '\n')
CatBoost_nri = nri(df_MRI_CatBoost, 'MRI_pred', 'CatBoost_pred', 'label')
print('\n')
CatBoost_nri.to_excel(
    r"E:\Pystudy\Thyroid_P_N\data\ML\NPM\emseble\em_code\results\Data_set\10.39\NRI_IDI\NRI_MRI_CatBoost.xlsx")

In [ ]:
# pairwise comprision in single, ensemble and MRI models with FDR

models = {'Logit': clf_MRI, 'LR': lr, 'KNN': clf_KNN, 'SVM': clf_SVM, 'LDA': clf_LDA, 'GNB': clf_GNB, 'CART': clf_CART, 'RF': clf_RF, 'Bagging': bag, 'AdaBoost': ada, 'GBDT': clf_GBDT, 'XGBoost': XGB, 'LightGBM': lgb, 'CatBoost': cat}

aucs = {}
for name, model in models.items():
    if name == 'Logit':
        predict = model.predict(x_test_step1[independent_vars])
    else:
        predict = model.predict_proba(x_selected_test)[:, 1]
    aucs[name] = roc_auc_score(y_test, predict)

p_values = []
comparisons = []
for i, (name_1, auc_1) in enumerate(aucs.items()):
    for j in range(i+1, len(aucs)):
        name_2, auc_2 = list(aucs.items())[j]
        if name_1 == 'Logit':
            predict_1 = models[name_1].predict(x_test_step1[independent_vars])
        else:
            predict_1 = models[name_1].predict_proba(x_selected_test)[:, 1]
        if name_2 == 'Logit':
            predict_2 = models[name_2].predict(x_test_step1[independent_vars])
        else:
            predict_2 = models[name_2].predict_proba(x_selected_test)[:, 1]
        pvalue = DelongTest(predict_1, predict_2, y_test).p_value
        p_values.append(pvalue)
        comparisons.append((name_1, name_2, auc_1, auc_2, pvalue))

rejected, p_values_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

for i in range(len(comparisons)):
    comparisons[i] = comparisons[i][:4] + (p_values_corrected[i],)

df = pd.DataFrame(comparisons, columns=['Model 1', 'Model 2', 'AUC 1', 'AUC 2', 'Adjusted p-value'])
df.to_excel('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\MRI_single_em_roc.xlsx', index=False)

In [ ]:
# Heatmap of pairwise comprision in single, ensemble and MRI models with FDR

df_pvalues = pd.read_excel(
    r'E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\MRI_single_em_roc.xlsx')

clf_names = ['Logit', 'LR', 'KNN', 'SVM', 'LDA', 'GNB', 'CART', 'RF',
             'Bagging', 'AdaBoost', 'GBDT', 'XGBoost', 'LightGBM', 'CatBoost']
num_models = len(clf_names)
p_value_matrix = np.zeros((num_models, num_models))

for index, row in df_pvalues.iterrows():
    i = clf_names.index(row['Model 1'])
    j = clf_names.index(row['Model 2'])
    p_value_matrix[i, j] = row['Adjusted p-value']
    p_value_matrix[j, i] = row['Adjusted p-value']

fig, ax = plt.subplots()
cmap = plt.cm.YlGnBu
heatmap = ax.imshow(p_value_matrix, interpolation='nearest', cmap=cmap)

for i in range(len(clf_names)):
    for j in range(len(clf_names)):
        if i != j:
            p_val = p_value_matrix[i, j]
            if p_val < 0.001:
                display_text = "<0.001"
                color = 'red'
                weight = 'bold'
            elif p_val < 0.05:
                display_text = f"{p_val:.3f}"
                color = 'red'
                weight = 'bold'
            else:
                display_text = f"{p_val:.3f}"
                color = 'black'
                weight = 'normal'
            text = ax.text(j, i, display_text,
                           ha="center", va="center", color=color, fontweight=weight)

ax.set_xticks(np.arange(num_models))
ax.set_yticks(np.arange(num_models))
ax.set_xticklabels(clf_names, fontsize=10)
ax.set_yticklabels(clf_names, fontsize=10)
plt.xticks(rotation=45, ha='right', fontsize=10)
ax.set_title(
    'Adjusted p-values from Delong Test with FDR Correction', fontsize=12)

cbar = fig.colorbar(heatmap, ax=ax, shrink=0.56, aspect=30)
cbar.ax.set_ylabel('Adjusted p-value', fontsize=12)

plt.tight_layout()
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\single_em_MRI_heatmap_FDR_corrected.png', dpi=300)
plt.show()

In [ ]:
# Bar plot of single, ensemble and MRI models

aucs = {'Logit': clf_MRI_test_auc, 'LR': LR_Testing_AUC, 'KNN': KNN_Testing_AUC, 'SVM': SVM_Testing_AUC, 'LDA': LDA_Testing_AUC, 'GNB': GNB_Testing_AUC, 'CART': CART_Testing_AUC, 'RF':RF_Testing_AUC, 'Bagging':bag_Testing_AUC, 'AdaBoost':AdaBoost_Testing_AUC, 'GBDT':GBDT_Testing_AUC, 'XGBoost':XGB_Testing_AUC, 'LightGBM':lgb_Testing_AUC, 'CatBoost':cat_Testing_AUC, }
conf_intervals = {'Logit': clf_MRI_test_auc_CI, 'LR': LR_Testing_AUC_CI, 'KNN': KNN_Testing_AUC_CI, 'SVM': SVM_Testing_AUC_CI, 'LDA': LDA_Testing_AUC_CI, 'GNB': GNB_Testing_AUC_CI, 'CART': CART_Testing_AUC_CI, 'RF':RF_Testing_AUC_CI, 'Bagging':bag_Testing_AUC_CI, 'AdaBoost':AdaBoost_Testing_AUC_CI, 'GBDT':GBDT_Testing_AUC_CI, 'XGBoost':XGB_Testing_AUC_CI, 'LightGBM':lgb_Testing_AUC_CI, 'CatBoost':cat_Testing_AUC_CI}

import random
colors = []
for i in range(len(aucs)):
    r = lambda: random.randint(0, 255)
    colors.append('#%02X%02X%02X' % (r(), r(), r()))

fig, ax = plt.subplots()
ax.bar(
    aucs.keys(),
    aucs.values(),
    color=colors,
    yerr=[(conf_intervals[m][1] - conf_intervals[m][0]) / 2 for m in aucs.keys()],
    capsize=5,
)

ax.set_title('Logit, Single and Ensemble AUC Comparison', fontsize=12)
ax.set_xticklabels(aucs.keys(), rotation=45, ha='right', fontsize=10)
ax.set_ylabel('AUC', fontsize=12)

plt.title('Logit, Single and Ensemble Models', fontsize=12)
plt.savefig('E:\\Pystudy\\Thyroid_P_N\\data\\ML\\NPM\\emseble\\em_code\\results\\Data_set\\10.39\\figures\\Logit_Single_Ensemble_C.png', dpi = 300)
plt.show()